# SSI Complete Workflow - Issuer, Holder & Verifier

Dieses Notebook demonstriert den kompletten Self-Sovereign Identity Workflow mit drei ACA-Py Agenten:
- **Issuer Agent** (Port 8021): Gibt Credentials aus
- **Holder Agent** (Port 8031): Empfängt und speichert Credentials
- **Verifier Agent** (Port 8041): Verifiziert Presentations

## Voraussetzungen
1. VON-Network läuft: `cd ../von-network && ./manage start --wait`
2. Alle drei Agenten laufen: `docker-compose up -d`
3. Python-Pakete installiert: `pip install requests pandas matplotlib plotly networkx`

## Teil 1: Setup & Vorbereitung

In [ ]:
# Cell 1: Imports und Konfiguration
import requests
import json
import time
import pandas as pd
from IPython.display import display, Markdown, HTML
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# SSL Warnung für self-signed Zertifikate unterdrücken (PQC Proxy)
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Agent URLs
ISSUER_ADMIN_URL = "https://localhost:8021"
HOLDER_ADMIN_URL = "https://localhost:8031"
VERIFIER_ADMIN_URL = "https://localhost:8041"
VON_NETWORK_URL = "https://localhost:8000"  # PQC HTTPS Reverse Proxy

# Farben für Visualisierungen
ISSUER_COLOR = "#3498db"  # Blau
HOLDER_COLOR = "#2ecc71"  # Grün
VERIFIER_COLOR = "#e74c3c"  # Rot

print("✅ Imports erfolgreich")
print(f"📍 Issuer:   {ISSUER_ADMIN_URL}")
print(f"📍 Holder:   {HOLDER_ADMIN_URL}")
print(f"📍 Verifier: {VERIFIER_ADMIN_URL}")
print(f"📍 Ledger:   {VON_NETWORK_URL}")

# ========================================
# Tails-Server Konfiguration (für Revocation)
# ========================================
TAILS_SERVER_URL = "https://localhost:6543"
TAILS_FILE_COUNT = 100  # Max Credentials per Registry

print(f"📍 Tails:    {TAILS_SERVER_URL}")
print(f"   Max Credentials per Registry: {TAILS_FILE_COUNT}")

# ========================================
# Helper Functions
# ========================================

def api_get(base_url, path):
    """GET Request zu ACA-Py Admin API"""
    url = f"{base_url}{path}"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def api_post(base_url, path, data):
    """POST Request zu ACA-Py Admin API"""
    url = f"{base_url}{path}"
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=data, headers=headers)
    return response.json() if response.status_code == 200 else None

def pretty_print(data, title=""):
    """Pretty print JSON data"""
    if title:
        print(f"\n{'='*60}")
        print(f"  {title}")
        print('='*60)
    print(json.dumps(data, indent=2))
    print()

print("\n✅ Setup komplett!")


In [ ]:
# Cell 2: Helper-Funktionen für API-Calls

def api_get(url, path):
    """GET Request an ACA-Py Admin API"""
    try:
        response = requests.get(f"{url}{path}")
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ GET Fehler: {e}")
        return None

def api_post(url, path, data=None):
    """POST Request an ACA-Py Admin API"""
    try:
        response = requests.post(
            f"{url}{path}",
            json=data,
            headers={"Content-Type": "application/json"}
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ POST Fehler: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"   Response: {e.response.text}")
        return None

def pretty_print(data, title=""):
    """Formatierte JSON-Ausgabe"""
    if title:
        print(f"\n{'='*60}")
        print(f"  {title}")
        print(f"{'='*60}")
    print(json.dumps(data, indent=2))

def wait_for_status(url, path, field, expected_value, timeout=30, interval=1):
    """Wartet bis ein Feld einen bestimmten Wert hat"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        response = api_get(url, path)
        if response and response.get(field) == expected_value:
            return response
        time.sleep(interval)
    print(f"⚠️  Timeout: {field} erreichte nicht '{expected_value}' nach {timeout}s")
    return None

def status_badge(status, label="Status"):
    """HTML Badge für Status"""
    colors = {
        "active": "green",
        "completed": "green",
        "done": "green",
        "ready": "green",
        "pending": "orange",
        "failed": "red",
        "error": "red"
    }
    color = colors.get(status.lower(), "gray")
    return f'<span style="background-color:{color};color:white;padding:3px 8px;border-radius:3px;font-weight:bold">{label}: {status}</span>'

print("✅ Helper-Funktionen geladen")

In [ ]:
# Cell 3: Agenten-Status prüfen (Health Check)

def check_agent_status(name, url):
    """Prüft ob ein Agent bereit ist"""
    try:
        response = api_get(url, "/status/ready")
        if response and response.get("ready"):
            return f"✅ {name}: Bereit"
        else:
            return f"⚠️  {name}: Nicht bereit"
    except:
        return f"❌ {name}: Nicht erreichbar"

print("🔍 Agenten-Status Check...\n")
print(check_agent_status("Issuer", ISSUER_ADMIN_URL))
print(check_agent_status("Holder", HOLDER_ADMIN_URL))
print(check_agent_status("Verifier", VERIFIER_ADMIN_URL))

# Detaillierte Informationen mit Conductor-Statistiken
print("\n📊 Detaillierte Agent-Informationen:\n")
agents_info = []
for name, url in [("Issuer", ISSUER_ADMIN_URL), ("Holder", HOLDER_ADMIN_URL), ("Verifier", VERIFIER_ADMIN_URL)]:
    status = api_get(url, "/status")
    if status:
        conductor = status.get("conductor", {})
        agents_info.append({
            "Agent": name,
            "Label": status.get("label", "N/A"),
            "Version": status.get("version", "N/A"),
            "In Sessions": conductor.get("in_sessions", 0),
            "Out Encode": conductor.get("out_encode", 0),
            "Out Deliver": conductor.get("out_deliver", 0),
            "Active": conductor.get("task_active", 0),
            "Done": conductor.get("task_done", 0),
            "Failed": conductor.get("task_failed", 0),
            "Pending": conductor.get("task_pending", 0)
        })

df = pd.DataFrame(agents_info)
display(df)

In [ ]:
# Cell 4: VON-Network Verbindung testen

print("🔗 VON-Network Verbindung testen...\n")

try:
    # Genesis-Datei abrufen
    genesis_response = requests.get(f"{VON_NETWORK_URL}/genesis")
    if genesis_response.status_code == 200:
        print("✅ VON-Network erreichbar")
        print(f"   Genesis-Datei: {len(genesis_response.text)} Bytes")
    
    # Ledger-Status abrufen
    status_response = requests.get(f"{VON_NETWORK_URL}/status")
    if status_response.status_code == 200:
        print("✅ Ledger-Status abrufbar")
        
except Exception as e:
    print(f"❌ VON-Network nicht erreichbar: {e}")
    print("   Bitte starte VON-Network: cd ../von-network && ./manage start --wait")

In [ ]:
# Cell 5: Architektur visualisieren

display(Markdown("""
## 🏗️ Architektur-Übersicht

```
┌─────────────────────────────────────────────────────────────┐
│                    SSI Ecosystem                             │
├─────────────────────────────────────────────────────────────┤
│                                                               │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐  │
│  │   ISSUER     │───▶│    HOLDER    │◀───│  VERIFIER    │  │
│  │  (Faber)     │    │   (Alice)    │    │   (Acme)     │  │
│  │  Port: 8021  │    │  Port: 8031  │    │  Port: 8041  │  │
│  └──────┬───────┘    └──────┬───────┘    └──────┬───────┘  │
│         │                    │                    │          │
│         └────────────────────┼────────────────────┘          │
│                              │                                │
│                    ┌─────────▼─────────┐                     │
│                    │  VON Network      │                     │
│                    │  Hyperledger Indy │                     │
│                    │  Port: 9000       │                     │
│                    └───────────────────┘                     │
│                                                               │
└─────────────────────────────────────────────────────────────┘
```

**Workflow-Schritte:**
1. 🔑 **Ledger Setup**: DIDs auf Ledger registrieren
2. 📋 **Schema**: Issuer erstellt Schema auf Ledger
3. 🔐 **Cred Def**: Issuer erstellt Credential Definition
4. 🤝 **Connections**: Agenten verbinden sich untereinander
5. 📜 **Issue**: Issuer stellt Credential an Holder aus
6. ✅ **Verify**: Verifier verifiziert Presentation vom Holder
"""))

## Teil 2: Ledger-Verbindung & DIDs

In diesem Abschnitt erstellen und registrieren wir DIDs für Issuer und Verifier auf dem Ledger.
Der Holder benötigt keine Public DID, da er nur Peer-DIDs für Connections nutzt.

In [ ]:
# Cell 7: Performance Tracking Setup

# Dictionary um alle Performance-Metriken zu sammeln
performance_metrics = {
    "did_creation": [],
    "ledger_registration": [],
    "schema_creation": [],
    "cred_def_creation": [],
    "connection_establishment": [],
    "credential_issuance": [],
    "proof_presentation": []
}

def measure_time(operation_name):
    """Decorator für Performance-Messung"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            duration = end_time - start_time
            performance_metrics[operation_name].append(duration)
            print(f"⏱️  {operation_name}: {duration:.3f}s")
            return result
        return wrapper
    return decorator

print("✅ Performance Tracking aktiviert")

In [ ]:
# Cell 6: Issuer - DID erstellen

print("🔑 Issuer: DID erstellen...\n")

# DID-Erstellungs-Parameter für did:indy
# API: POST /did/indy/create
did_params = {
    "options": {
        "key_type": "ed25519",  # Kryptografischer Key-Type (EdDSA)
        "seed": "000000000000000000000000Issuer01"  # 32 Zeichen für reproduzierbare DID
    }
}

print(f"📋 DID-Parameter:")
print(f"   Method:   indy")
print(f"   Endpoint: /did/indy/create")
print(f"   Key Type: {did_params['options']['key_type']}")
print(f"   Seed:     {'(zufällig)' if 'seed' not in did_params['options'] else did_params['options']['seed']}")
print(f"   Format:   did:indy:<identifier>\n")

# DID erstellen mit korrektem Endpunkt
issuer_did_response = api_post(
    ISSUER_ADMIN_URL,
    "/did/indy/create",  # Korrekter Endpunkt für did:indy
    did_params
)

if issuer_did_response is not None:
    # Response-Format: {"did": "did:indy:...", "verkey": "..."}
    issuer_did = issuer_did_response.get("did")
    issuer_verkey = issuer_did_response.get("verkey")
    
    print(f"✅ Issuer DID erstellt:")
    print(f"   DID:    {issuer_did}")
    print(f"   Verkey: {issuer_verkey}")
    
    # Zeige vollständige Response
    pretty_print(issuer_did_response, "DID-Erstellungs-Response (Issuer)")
    
    # Speichern für spätere Verwendung
    issuer_info = {
        "did": issuer_did,
        "verkey": issuer_verkey,
        "method": "indy",
        "key_type": did_params["options"]["key_type"],
        "role": "ENDORSER"
    }
    
    print(f"\n💡 Format: {issuer_did} (vollständige qualified DID)")
else:
    print("❌ Fehler beim Erstellen der Issuer DID")

In [ ]:
# Cell 8: Issuer DID auf Ledger registrieren

print("📝 Issuer DID auf Ledger registrieren (via VON-Network)...\n")

start_time = time.time()

# Registrierungs-Daten für VON-Network (DID-Based Registration)
# Nutzt DID+Verkey aus Cell 6 statt Seed
register_data = {
    "did": issuer_did,        # DID aus Cell 6 (ACA-Py Wallet)
    "verkey": issuer_verkey,  # Verkey aus Cell 6 (ACA-Py Wallet)
    "alias": "Issuer Agent",
    "role": "ENDORSER"  # TRUST_ANCHOR oder ENDORSER für Issuer-Rechte
}

print(f"📋 Registrierungs-Parameter:")
print(f"   Ledger URL: {VON_NETWORK_URL}/register")
print(f"   DID:        {register_data['did']}")
print(f"   Verkey:     {register_data['verkey'][:20]}...")
print(f"   Alias:      {register_data['alias']}")
print(f"   Role:       {register_data['role']}\n")

# Registriere DID auf VON-Network Ledger
# Wie in demo code: resp = await self.client_session.post(von_network_url + "/register", json=data)
print(f"🌐 POST {VON_NETWORK_URL}/register")

try:
    response = requests.post(
        f"{VON_NETWORK_URL}/register",
        json=register_data,
        timeout=10
    )
    
    response.raise_for_status()
    
    # Response format: {"did": "did:indy:...", "verkey": "..."}
    nym_info = response.json()
    
    issuer_did = nym_info["did"]
    issuer_verkey = nym_info["verkey"]
    
    duration = time.time() - start_time
    performance_metrics["ledger_registration"].append(duration)
    
    print(f"✅ Issuer DID auf Ledger registriert")
    print(f"   DID:    {issuer_did}")
    print(f"   Verkey: {issuer_verkey}")
    print(f"⏱️  Zeit: {duration:.3f}s")
    
    # Zeige vollständige Ledger-Response
    pretty_print(nym_info, "VON-Network Registration Response (Issuer)")
    
    # JETZT: Setze DID als Public DID im ACA-Py Wallet
    # Wie in demo code: await self.admin_POST("/wallet/did/public?did=" + self.did)
    print(f"\n🔑 Setze DID als Public DID im Wallet...")
    public_did_response = api_post(
        ISSUER_ADMIN_URL,
        f"/wallet/did/public?did={issuer_did}"
    )
    
    if public_did_response is not None:
        print(f"✅ Issuer DID als Public DID gesetzt")
    else:
        print(f"⚠️  Warnung: Public DID konnte nicht gesetzt werden")
    
    # Speichern für spätere Verwendung
    issuer_info = {
        "did": issuer_did,
        "verkey": issuer_verkey,
        "method": "indy",
        "key_type": "ed25519",
        "role": "ENDORSER"
    }
    
    # Hole NYM Role vom Ledger zur Verifikation
    print(f"\n🔍 NYM Role vom Ledger verifizieren...")
    nym_role_response = api_get(
        ISSUER_ADMIN_URL,
        f"/ledger/get-nym-role?did={issuer_did}"
    )
    
    if nym_role_response is not None:
        print(f"✅ NYM Role vom Ledger abgerufen")
        pretty_print(nym_role_response, "Ledger NYM Role (Issuer DID)")
        
        # Extrahiere Role
        role = nym_role_response.get('role', 'N/A')
        print(f"\n📋 Verifizierte Rolle: {role}")
    else:
        print(f"⚠️  NYM Role konnte nicht abgerufen werden")
        
except requests.exceptions.RequestException as e:
    print(f"❌ POST Fehler: {e}")
    print(f"   Stelle sicher dass VON-Network läuft: cd von-network && ./manage start")


In [ ]:
# Cell 9: Verifier - DID erstellen und registrieren

print("🔑 Verifier: DID erstellen und registrieren...\n")

start_time = time.time()

# DID-Erstellungs-Parameter für did:indy
# API: POST /did/indy/create
did_params = {
    "options": {
        "key_type": "ed25519"  # Kryptografischer Key-Type (EdDSA)
        # "seed": "00000000000000000000000Verifier1"  # Optional: 32 Zeichen für reproduzierbare DIDs
    }
}

print(f"📋 DID-Parameter:")
print(f"   Method:   indy")
print(f"   Endpoint: /did/indy/create")
print(f"   Key Type: {did_params['options']['key_type']}")
print(f"   Seed:     {'(zufällig)' if 'seed' not in did_params['options'] else did_params['options']['seed']}")
print(f"   Format:   did:indy:<identifier>\n")

# DID erstellen mit korrektem Endpunkt
verifier_did_response = api_post(
    VERIFIER_ADMIN_URL,
    "/did/indy/create",  # Korrekter Endpunkt für did:indy
    did_params
)

if verifier_did_response is not None:
    # Response-Format: {"did": "did:indy:...", "verkey": "..."}
    verifier_did = verifier_did_response.get("did")
    verifier_verkey = verifier_did_response.get("verkey")
    
    duration_create = time.time() - start_time
    performance_metrics["did_creation"].append(duration_create)
    
    print(f"✅ Verifier DID erstellt:")
    print(f"   DID:    {verifier_did}")
    print(f"   Verkey: {verifier_verkey}")
    print(f"⏱️  Zeit: {duration_create:.3f}s\n")
    
    # Registriere auf VON-Network Ledger
    # Wie in demo code: resp = await self.client_session.post(von_network_url + "/register", json=data)
    print("📝 Registriere NYM auf VON-Network Ledger...")
    start_time = time.time()
    
    register_data = {
        "did": verifier_did,
        "verkey": verifier_verkey,
        "alias": "Verifier Agent",
        "role": "ENDORSER"
    }
    
    print(f"📋 Registrierungs-Parameter:")
    print(f"   Ledger URL: {VON_NETWORK_URL}/register")
    print(f"   DID:        {verifier_did}")
    print(f"   Alias:      {register_data['alias']}")
    print(f"   Role:       {register_data['role']}\n")
    
    print(f"🌐 POST {VON_NETWORK_URL}/register")
    
    try:
        response = requests.post(
            f"{VON_NETWORK_URL}/register",
            json=register_data,
            timeout=10
        )
        
        response.raise_for_status()
        
        nym_info = response.json()
        
        duration_register = time.time() - start_time
        performance_metrics["ledger_registration"].append(duration_register)
        
        print(f"✅ Verifier DID auf Ledger registriert")
        print(f"⏱️  Zeit: {duration_register:.3f}s")
        
        # Zeige vollständige Ledger-Response
        pretty_print(nym_info, "VON-Network Registration Response (Verifier)")
        
        # JETZT: Setze DID als Public DID im ACA-Py Wallet
        print(f"\n🔑 Setze DID als Public DID im Wallet...")
        public_did_response = api_post(
            VERIFIER_ADMIN_URL,
            f"/wallet/did/public?did={verifier_did}"
        )
        
        if public_did_response is not None:
            print(f"✅ Verifier DID als Public DID gesetzt")
        else:
            print(f"⚠️  Warnung: Public DID konnte nicht gesetzt werden")
        
        verifier_info = {
            "did": verifier_did,
            "verkey": verifier_verkey,
            "method": "indy",
            "key_type": did_params["options"]["key_type"],
            "role": "ENDORSER"
        }
        
        # Warte kurz damit die Transaction auf dem Ledger verfügbar ist
        time.sleep(1)
        
        # Hole NYM Role vom Ledger zur Verifikation
        print(f"\n🔍 NYM Role vom Ledger verifizieren...")
        nym_role_response = api_get(
            VERIFIER_ADMIN_URL,
            f"/ledger/get-nym-role?did={verifier_did}"
        )
        
        if nym_role_response is not None:
            print(f"✅ NYM Role vom Ledger abgerufen")
            pretty_print(nym_role_response, "Ledger NYM Role (Verifier DID)")
            
            # Extrahiere Role
            role = nym_role_response.get('role', 'N/A')
            print(f"\n📋 Verifizierte Rolle: {role}")
        else:
            print(f"⚠️  NYM Role konnte nicht abgerufen werden")
        
        print(f"\n💡 Format: {verifier_did} (vollständige qualified DID)")
        
    except requests.exceptions.RequestException as e:
        print(f"❌ POST Fehler: {e}")
        print(f"   Stelle sicher dass VON-Network läuft: cd von-network && ./manage start")
else:
    print("❌ Fehler beim Erstellen der Verifier DID")

In [ ]:
# Cell 10a: Domain Ledger Transaktionen

print("📋 Domain Ledger - Letzte Transaktionen")
print("🔒 Verbindung über PQC HTTPS Reverse Proxy\n")

try:
    # Hole alle Domain Ledger Transaktionen vom VON-Network
    ledger_response = requests.get(f"{VON_NETWORK_URL}/ledger/domain")
    
    if ledger_response.status_code == 200:
        # FIX: Response ist ein paginiertes Objekt, nicht direkt ein Array
        ledger_data = ledger_response.json()
        ledger_txns = ledger_data.get('results', [])
        total_txns = ledger_data.get('total', 0)
        page = ledger_data.get('page', 1)
        page_size = ledger_data.get('page_size', 100)
        
        print(f"✅ Domain Ledger abgerufen")
        print(f"   Total Transaktionen: {total_txns}")
        print(f"   Seite: {page} (Page Size: {page_size})\n")
        
        # Zeige die letzten 10 Transaktionen
        recent_txns = ledger_txns[-10:] if len(ledger_txns) > 10 else ledger_txns
        
        print("📊 Letzte 10 Transaktionen:\n")
        txn_summary = []
        
        for txn in recent_txns:
            # FIX: Korrekte verschachtelte Struktur-Zugriffe
            txn_metadata = txn.get('txnMetadata', {})
            txn_data = txn.get('txn', {})
            txn_type = txn_data.get('type', 'N/A')
            
            # Transaction Type Namen
            type_names = {
                '1': 'NYM',
                '100': 'ATTRIB',
                '101': 'SCHEMA',
                '102': 'CRED_DEF',
                '113': 'REVOC_REG_DEF',
                '114': 'REVOC_REG_ENTRY'
            }
            
            type_name = type_names.get(str(txn_type), f"Type {txn_type}")
            
            # FIX: Extrahiere 'from' aus txn.metadata (nicht txn_metadata)
            requester = txn_data.get('metadata', {}).get('from', 'N/A')
            #requester_short = requester[:20] + '...' if len(requester) > 20 else requester
            
            txn_summary.append({
                "Seq No": txn_metadata.get('seqNo', 'N/A'),
                "Type": type_name,
                "Transaction Time": txn_metadata.get('txnTime', 'N/A'),
                "Requester": requester
            })
        
        if txn_summary:
            df_txns = pd.DataFrame(txn_summary)
            display(df_txns)
        
        # Detaillierte Ansicht der letzten Transaction
        if recent_txns:
            print("\n🔍 Detaillierte Ansicht der letzten Transaction:")
            pretty_print(recent_txns[-1], "Letzte Ledger-Transaction")
    else:
        print(f"⚠️  Konnte Ledger nicht abrufen: {ledger_response.status_code}")
        
except Exception as e:
    print(f"❌ Fehler beim Abrufen des Ledgers: {e}")

print("\n💡 Transaction Types:")
print("   • NYM (1)           = DID Registration")
print("   • ATTRIB (100)      = DID Attribute")
print("   • SCHEMA (101)      = Schema Definition")
print("   • CRED_DEF (102)    = Credential Definition")
print("   • REVOC_REG_* (113/114) = Revocation Registry")

In [ ]:
# Cell 10: Ledger-Übersicht - Alle DIDs anzeigen

print("📋 Ledger-Übersicht - Alle DIDs:\n")

# Alle DIDs von allen Agenten abrufen
issuer_dids = api_get(ISSUER_ADMIN_URL, "/wallet/did")
holder_dids = api_get(HOLDER_ADMIN_URL, "/wallet/did")
verifier_dids = api_get(VERIFIER_ADMIN_URL, "/wallet/did")

# Zeige originale Responses
print("🔍 Originale /wallet/did Responses:\n")
if issuer_dids:
    pretty_print(issuer_dids, "Issuer Wallet DIDs")
if holder_dids:
    pretty_print(holder_dids, "Holder Wallet DIDs")
if verifier_dids:
    pretty_print(verifier_dids, "Verifier Wallet DIDs")

# Erstelle erweiterte Übersicht
print("\n📊 DID-Übersicht (Detailliert):\n")
ledger_overview = []

# Issuer DIDs
if issuer_dids and "results" in issuer_dids:
    for did_info in issuer_dids["results"]:
        posture = did_info.get("posture", "unknown")
        # Zeige alle DIDs (posted und wallet_only)
        ledger_overview.append({
            "Agent": "Issuer",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Holder DIDs
if holder_dids and "results" in holder_dids:
    for did_info in holder_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Holder",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Verifier DIDs
if verifier_dids and "results" in verifier_dids:
    for did_info in verifier_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Verifier",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Als DataFrame anzeigen
if ledger_overview:
    df_ledger = pd.DataFrame(ledger_overview)
    display(df_ledger)
else:
    print("⚠️  Keine DIDs gefunden")

# Zusammenfassung
print("\n💡 Posture-Bedeutung:")
print("   • posted       = DID auf Ledger registriert und veröffentlicht")
print("   • public       = DID als Public DID gesetzt")
print("   • wallet_only  = DID nur im Wallet, nicht auf Ledger")
print("\n💡 Hinweis:")
print("   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef")
print("   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen")
print("   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen")

## Teil 3: Schema & Credential Definition

Der Issuer erstellt jetzt ein Schema auf dem Ledger und eine Credential Definition, die für die Ausstellung von Credentials benötigt wird.

In [ ]:
# Cell 11: Schema erstellen (Indy)

print("📋 Schema erstellen...\n")

start_time = time.time()

# Schema-Daten für Indy API
# API: POST /schemas
# Format: {"schema_name": "...", "schema_version": "...", "attributes": [...]}
schema_data = {
    "schema_name": "university_degree",
    "schema_version": "1.0",
    "attributes": [
        "name",
        "degree",
        "university",
        "graduation_date",
        "age",
        "gpa"
    ]
}

print(f"📋 Schema-Parameter (Indy):")
print(f"   Endpoint:  /schemas")
print(f"   Name:      {schema_data['schema_name']}")
print(f"   Version:   {schema_data['schema_version']}")
print(f"   Attribute: {', '.join(schema_data['attributes'])}\n")

# Schema auf Ledger registrieren (Indy endpoint)
schema_response = api_post(
    ISSUER_ADMIN_URL,
    "/schemas",  # Indy endpoint
    schema_data
)

if schema_response is not None:
    # Response Format: {"schema_id": "...", "schema": {...}}
    schema_id = schema_response.get("schema_id")
    
    duration = time.time() - start_time
    performance_metrics["schema_creation"].append(duration)
    
    print(f"✅ Schema erstellt (Indy)")
    print(f"   Schema ID: {schema_id}")
    print(f"⏱️  Zeit: {duration:.3f}s")
    
    # Zeige vollständige Schema-Response
    pretty_print(schema_response, "Indy Schema Response")
    
else:
    print("❌ Fehler beim Erstellen des Schemas")


In [ ]:
# Cell 12: Credential Definition erstellen (Indy)

print("\n🔐 Credential Definition erstellen...\n")

start_time = time.time()

# Credential Definition Daten für Indy API
# API: POST /credential-definitions
# Format: {"schema_id": "...", "tag": "...", "support_revocation": ...}
cred_def_data = {
    "schema_id": schema_id,    # Schema ID aus Cell 11
    "tag": "default",
    "support_revocation": True,
    "revocation_registry_size": TAILS_FILE_COUNT
}

print(f"📋 Credential Definition Parameter (Indy):")
print(f"   Endpoint:           /credential-definitions")
print(f"   Schema ID:          {cred_def_data['schema_id']}")
print(f"   Tag:                {cred_def_data['tag']}")
print(f"   Support Revocation: {cred_def_data['support_revocation']}")
print(f"   Support Revocation: {cred_def_data['revocation_registry_size']}\n")

# Credential Definition auf Ledger registrieren (Indy endpoint)
cred_def_response = api_post(
    ISSUER_ADMIN_URL,
    "/credential-definitions",  # Indy endpoint
    cred_def_data
)

if cred_def_response is not None:
    # Response Format: {"credential_definition_id": "...", "credential_definition": {...}}
    cred_def_id = cred_def_response.get("credential_definition_id")
    
    duration = time.time() - start_time
    performance_metrics["cred_def_creation"].append(duration)
    
    print(f"✅ Credential Definition erstellt (Indy)")
    print(f"   Cred Def ID: {cred_def_id}")
    print(f"⏱️  Zeit: {duration:.3f}s")
    
    # Zeige vollständige Cred Def Response
    pretty_print(cred_def_response, "Indy Credential Definition Response")
    
    # Info: Indy Format
    print(f"\n💡 Indy Format:")
    print(f"   • Request:  {{\"schema_id\": ..., \"tag\": ..., \"support_revocation\": ...}}")
    print(f"   • Response: {{\"credential_definition_id\": ..., \"credential_definition\": {{...}}}}")
    print(f"   • Cred Def ID direkt aus Response")
    print(f"   • support_revocation ist top-level Parameter")
    
    print(f"\n📝 Nächste Schritte:")
    print(f"   • Schema ID:   {schema_id}")
    print(f"   • Cred Def ID: {cred_def_id}")
    print(f"   → Bereit für Connection und Credential Issuance")
else:
    print("❌ Fehler beim Erstellen der Credential Definition")


## Teil 4: Connections zwischen Agenten

Jetzt verbinden wir die Agenten untereinander:
1. **Issuer ↔ Holder**: Für Credential-Ausstellung
2. **Holder ↔ Verifier**: Für Proof-Präsentation

In [ ]:
# Cell 13: Connection Issuer → Holder erstellen

print("🤝 Connection: Issuer → Holder erstellen...\n")

start_time = time.time()

# ========================================
# 1. PRE-CHECK: Existierende Connections prüfen
# ========================================
print("🔍 Pre-Check: Prüfe existierende Connections...\n")

existing_conn_issuer_holder = None
existing_conn_holder = None

# Prüfe Issuer-Seite: GET /connections
issuer_conns_response = api_get(ISSUER_ADMIN_URL, "/connections")
if issuer_conns_response and "results" in issuer_conns_response:
    print(f"   Issuer: {len(issuer_conns_response['results'])} Connection(s) gefunden")
    for conn in issuer_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_issuer_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_issuer_holder} (State: completed)")
            break
else:
    print("   Issuer: Keine Connections gefunden")

# Prüfe Holder-Seite: GET /connections
holder_conns_response = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns_response and "results" in holder_conns_response:
    print(f"   Holder: {len(holder_conns_response['results'])} Connection(s) gefunden")
    for conn in holder_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_holder} (State: completed)")
            break
else:
    print("   Holder: Keine Connections gefunden")

if existing_conn_issuer_holder and existing_conn_holder:
    print(f"\n✅ Existierende Connection gefunden!")
    print(f"   Issuer Connection ID: {existing_conn_issuer_holder}")
    print(f"   Holder Connection ID: {existing_conn_holder}")
    print("   Überspringe Connection-Erstellung\n")
    conn_id_issuer_holder = existing_conn_issuer_holder
    conn_id_holder = existing_conn_holder
else:
    print("\n   Keine vollständige existierende Connection gefunden")
    print("   Erstelle neue Connection mit did:peer:2...\n")
    
    # ========================================
    # 2. HAUPT-WORKFLOW: Connection mit did:peer erstellen
    # ========================================
    
    # Initialisiere Variablen
    conn_id_issuer_holder = None
    conn_id_holder = None
    invitation_msg_id = None
    invitation_key = None
    
    # Issuer erstellt Out-of-Band Invitation mit did:peer:2
    invitation_data = {
        "handshake_protocols": ["https://didcomm.org/didexchange/1.1"],
        "use_did_method": "did:peer:4",
        "my_label": "Issuer Agent - University Credentials",
        "goal": "Establish connection for credential issuance",
        "goal_code": "issue-vc",
        "accept": [
            "didcomm/aip1",
            "didcomm/aip2;env=rfc19"
        ]
    }
    
    print("📋 Invitation-Modus: did:peer:2 (P2P Connection)")
    
    # API Call mit Query-Parametern für auto-accept
    invitation_response = api_post(
        ISSUER_ADMIN_URL,
        "/out-of-band/create-invitation?auto_accept=true&multi_use=false",
        invitation_data
    )
    
    if invitation_response and "invitation" in invitation_response:
        invitation = invitation_response["invitation"]
        oob_id = invitation_response.get("oob_id")
        invitation_url = invitation_response.get("invitation_url", "")
        state = invitation_response.get("state")
        
        # WICHTIG: Speichere invitation_msg_id aus der Invitation
        invitation_msg_id = invitation.get("@id")
        
        # Optional: Extrahiere invitation_key aus services (falls vorhanden)
        services = invitation.get("services", [])
        if services and len(services) > 0:
            # Bei did:peer:2 ist der Service oft inline
            service = services[0]
            if isinstance(service, dict):
                recipient_keys = service.get("recipientKeys", [])
                if recipient_keys:
                    invitation_key = recipient_keys[0]
        
        print(f"✅ Issuer: Invitation erstellt")
        print(f"   OOB ID: {oob_id}")
        print(f"   State: {state}")
        print(f"   Invitation Msg ID: {invitation_msg_id}")
        if invitation_key:
            print(f"   Invitation Key: {invitation_key[:30]}...")
        print(f"   Services: {len(invitation.get('services', []))} service(s)")
        print(f"   Label: {invitation_data['my_label']}")
        print(f"   Goal: {invitation_data['goal']}")
        
        # Holder akzeptiert Invitation (mit auto-accept)
        receive_response = api_post(
            HOLDER_ADMIN_URL,
            "/out-of-band/receive-invitation?auto_accept=true",
            invitation
        )
        
        if receive_response is not None:
            conn_id_holder = receive_response.get("connection_id")
            print(f"\n✅ Holder: Invitation akzeptiert")
            print(f"   Connection ID (Holder): {conn_id_holder}")
            
            # Issuer findet seine Connection via invitation_msg_id
            print(f"\n   Issuer sucht Connection via invitation_msg_id...")
            time.sleep(2)  # Kurze Wartezeit für DIDExchange Protocol
            
            # Hole alle Issuer Connections
            issuer_conns = api_get(ISSUER_ADMIN_URL, "/connections")
            if issuer_conns and "results" in issuer_conns:
                connections = issuer_conns["results"]
                
                print(f"     Prüfe {len(connections)} Connection(s)...")
                
                # Finde Connection anhand invitation_msg_id (exaktes Match!)
                connection_found = False
                for conn in connections:
                    conn_invitation_msg_id = conn.get("invitation_msg_id")
                    conn_invitation_key = conn.get("invitation_key")
                    
                    # Match via invitation_msg_id (bevorzugt)
                    if invitation_msg_id and conn_invitation_msg_id == invitation_msg_id:
                        conn_id_issuer_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Issuer: Connection gefunden via invitation_msg_id!")
                        print(f"      • Connection ID: {conn_id_issuer_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Msg ID: {conn_invitation_msg_id}")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        print(f"      • Updated At: {conn.get('updated_at', 'N/A')}")
                        connection_found = True
                        break
                    
                    # Fallback: Match via invitation_key
                    elif invitation_key and conn_invitation_key == invitation_key:
                        conn_id_issuer_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Issuer: Connection gefunden via invitation_key!")
                        print(f"      • Connection ID: {conn_id_issuer_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Key: {conn_invitation_key[:30]}...")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        connection_found = True
                        break
                
                if not connection_found:
                    print(f"     ⚠️  Keine Connection mit matching invitation_msg_id gefunden")
                    print(f"     Gesuchte invitation_msg_id: {invitation_msg_id}")
            else:
                print(f"     ❌ Konnte Issuer Connections nicht abrufen")
        else:
            print("❌ Holder konnte Invitation nicht akzeptieren")
    else:
        print("❌ Fehler beim Erstellen der Invitation")

# ========================================
# 3. POST-VALIDATION: Connection-Status verifizieren
# ========================================
print("\n🔍 Post-Validation: Verifiziere Connection-Status...\n")

validation_success = False
issuer_state = None
holder_state = None

if conn_id_issuer_holder:
    # Prüfe Issuer-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    issuer_conn_detail = api_get(
        ISSUER_ADMIN_URL,
        f"/connections/{conn_id_issuer_holder}"
    )
    
    if issuer_conn_detail:
        issuer_state = issuer_conn_detail.get("state")
        issuer_their_label = issuer_conn_detail.get("their_label")
        issuer_their_role = issuer_conn_detail.get("their_role")
        issuer_invitation_msg_id = issuer_conn_detail.get("invitation_msg_id")
        
        print(f"   📋 Issuer Connection {conn_id_issuer_holder}:")
        print(f"      • State: {issuer_state}")
        print(f"      • Their Label: {issuer_their_label}")
        print(f"      • Their Role: {issuer_their_role}")
        print(f"      • Invitation Msg ID: {issuer_invitation_msg_id}")

# Prüfe Holder-Seite (falls conn_id_holder bekannt)
if conn_id_holder:
    # Prüfe Holder-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    holder_conn_detail = api_get(
        HOLDER_ADMIN_URL,
        f"/connections/{conn_id_holder}"
    )
    
    if holder_conn_detail:
        holder_state = holder_conn_detail.get("state")
        holder_their_label = holder_conn_detail.get("their_label")
        holder_their_role = holder_conn_detail.get("their_role")
        holder_invitation_msg_id = holder_conn_detail.get("invitation_msg_id")
        
        print(f"\n   📋 Holder Connection {conn_id_holder}:")
        print(f"      • State: {holder_state}")
        print(f"      • Their Label: {holder_their_label}")
        print(f"      • Their Role: {holder_their_role}")
        print(f"      • Invitation Msg ID: {holder_invitation_msg_id}")
        
        # Verifiziere dass invitation_msg_id übereinstimmt
        if invitation_msg_id and issuer_invitation_msg_id == holder_invitation_msg_id:
            print(f"      ✅ Invitation Msg IDs stimmen überein!")

# Validierung
if issuer_state == "active" and holder_state == "active":
    validation_success = True
elif issuer_state == "active" and not conn_id_holder:
    # Bei existierender Connection nur Issuer-Seite bekannt
    validation_success = True

duration = time.time() - start_time
performance_metrics["connection_establishment"].append(duration)

# Finale Ausgabe
print(f"\n{'='*60}")
if validation_success:
    print(f"✅ CONNECTION ESTABLISHED (did:peer:2): Issuer ↔ Holder")
    print(f"   Issuer Connection ID: {conn_id_issuer_holder}")
    if conn_id_holder:
        print(f"   Holder Connection ID: {conn_id_holder}")
    print(f"   Issuer State: {issuer_state}")
    if holder_state:
        print(f"   Holder State: {holder_state}")
    if invitation_msg_id:
        print(f"   Invitation Msg ID: {invitation_msg_id}")
else:
    print(f"⚠️  CONNECTION STATUS UNCLEAR")
    print(f"   Issuer State: {issuer_state if issuer_state else 'N/A'}")
    print(f"   Holder State: {holder_state if holder_state else 'N/A'}")
    if conn_id_issuer_holder:
        print(f"   Issuer Connection ID: {conn_id_issuer_holder}")
    if conn_id_holder:
        print(f"   Holder Connection ID: {conn_id_holder}")

print(f"⏱️  Gesamtzeit: {duration:.3f}s")
print(f"{'='*60}")


In [ ]:
# Cell 14: Connection Holder → Verifier erstellen

print("🤝 Connection: Holder → Verifier erstellen...\n")

start_time = time.time()

# ========================================
# 1. PRE-CHECK: Existierende Connections prüfen
# ========================================
print("🔍 Pre-Check: Prüfe existierende Connections...\n")

existing_conn_verifier_holder = None
existing_conn_holder_verifier = None

# Prüfe Verifier-Seite: GET /connections
verifier_conns_response = api_get(VERIFIER_ADMIN_URL, "/connections")
if verifier_conns_response and "results" in verifier_conns_response:
    print(f"   Verifier: {len(verifier_conns_response['results'])} Connection(s) gefunden")
    for conn in verifier_conns_response["results"]:
        if conn.get("state") == "completed":
            existing_conn_verifier_holder = conn["connection_id"]
            print(f"     → Connection {existing_conn_verifier_holder} (State: completed)")
            break
else:
    print("   Verifier: Keine Connections gefunden")

# Prüfe Holder-Seite: GET /connections (für Holder→Verifier)
holder_conns_response = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns_response and "results" in holder_conns_response:
    print(f"   Holder: {len(holder_conns_response['results'])} Connection(s) gefunden")
    # Finde Connection zum Verifier (basierend auf Label oder neueste)
    holder_verifier_conns = [c for c in holder_conns_response["results"] if c.get("their_label") == "Verifier Agent"]
    if holder_verifier_conns:
        for conn in holder_verifier_conns:
            if conn.get("state") == "completed":
                existing_conn_holder_verifier = conn["connection_id"]
                print(f"     → Connection {existing_conn_holder_verifier} (State: completed, Partner: Verifier)")
                break
else:
    print("   Holder: Keine Connections gefunden")

if existing_conn_verifier_holder and existing_conn_holder_verifier:
    print(f"\n✅ Existierende Connection gefunden!")
    print(f"   Verifier Connection ID: {existing_conn_verifier_holder}")
    print(f"   Holder Connection ID: {existing_conn_holder_verifier}")
    print("   Überspringe Connection-Erstellung\n")
    conn_id_verifier_holder = existing_conn_verifier_holder
    conn_id_holder_verifier = existing_conn_holder_verifier
else:
    print("\n   Keine vollständige existierende Connection gefunden")
    print("   Erstelle neue Connection mit did:peer:2...\n")
    
    # ========================================
    # 2. HAUPT-WORKFLOW: Connection mit did:peer erstellen
    # ========================================
    
    # Initialisiere Variablen
    conn_id_verifier_holder = None
    conn_id_holder_verifier = None
    invitation_msg_id = None
    invitation_key = None
    
    # Verifier erstellt Out-of-Band Invitation mit did:peer:2
    invitation_data = {
        "handshake_protocols": ["https://didcomm.org/didexchange/1.1"],
        "use_did_method": "did:peer:4",
        "my_label": "Verifier Agent - Credential Verification",
        "goal": "Establish connection for credential verification",
        "goal_code": "verify-vc",
        "accept": [
            "didcomm/aip1",
            "didcomm/aip2;env=rfc19"
        ]
    }
    
    print("📋 Invitation-Modus: did:peer:2 (P2P Connection)")
    
    # API Call mit Query-Parametern für auto-accept
    invitation_response = api_post(
        VERIFIER_ADMIN_URL,
        "/out-of-band/create-invitation?auto_accept=true&multi_use=false",
        invitation_data
    )
    
    if invitation_response and "invitation" in invitation_response:
        invitation = invitation_response["invitation"]
        oob_id = invitation_response.get("oob_id")
        invitation_url = invitation_response.get("invitation_url", "")
        state = invitation_response.get("state")
        
        # WICHTIG: Speichere invitation_msg_id aus der Invitation
        invitation_msg_id = invitation.get("@id")
        
        # Optional: Extrahiere invitation_key aus services (falls vorhanden)
        services = invitation.get("services", [])
        if services and len(services) > 0:
            service = services[0]
            if isinstance(service, dict):
                recipient_keys = service.get("recipientKeys", [])
                if recipient_keys:
                    invitation_key = recipient_keys[0]
        
        print(f"✅ Verifier: Invitation erstellt")
        print(f"   OOB ID: {oob_id}")
        print(f"   State: {state}")
        print(f"   Invitation Msg ID: {invitation_msg_id}")
        if invitation_key:
            print(f"   Invitation Key: {invitation_key}")
        print(f"   Services: {len(invitation.get('services', []))} service(s)")
        print(f"   Label: {invitation_data['my_label']}")
        print(f"   Goal: {invitation_data['goal']}")
        
        # Holder akzeptiert Invitation (mit auto-accept)
        receive_response = api_post(
            HOLDER_ADMIN_URL,
            "/out-of-band/receive-invitation?auto_accept=true",
            invitation
        )
        
        if receive_response is not None:
            conn_id_holder_verifier = receive_response.get("connection_id")
            print(f"\n✅ Holder: Invitation akzeptiert")
            print(f"   Connection ID (Holder): {conn_id_holder_verifier}")
            
            # Verifier findet seine Connection via invitation_msg_id
            print(f"\n   Verifier sucht Connection via invitation_msg_id...")
            time.sleep(2)  # Kurze Wartezeit für DIDExchange Protocol
            
            # Hole alle Verifier Connections
            verifier_conns = api_get(VERIFIER_ADMIN_URL, "/connections")
            if verifier_conns and "results" in verifier_conns:
                connections = verifier_conns["results"]
                
                print(f"     Prüfe {len(connections)} Connection(s)...")
                
                # Finde Connection anhand invitation_msg_id (exaktes Match!)
                connection_found = False
                for conn in connections:
                    conn_invitation_msg_id = conn.get("invitation_msg_id")
                    conn_invitation_key = conn.get("invitation_key")
                    
                    # Match via invitation_msg_id (bevorzugt)
                    if invitation_msg_id and conn_invitation_msg_id == invitation_msg_id:
                        conn_id_verifier_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Verifier: Connection gefunden via invitation_msg_id!")
                        print(f"      • Connection ID: {conn_id_verifier_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Msg ID: {conn_invitation_msg_id}")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        print(f"      • Updated At: {conn.get('updated_at', 'N/A')}")
                        connection_found = True
                        break
                    
                    # Fallback: Match via invitation_key
                    elif invitation_key and conn_invitation_key == invitation_key:
                        conn_id_verifier_holder = conn["connection_id"]
                        conn_state = conn.get("state")
                        print(f"\n   ✅ Verifier: Connection gefunden via invitation_key!")
                        print(f"      • Connection ID: {conn_id_verifier_holder}")
                        print(f"      • State: {conn_state}")
                        print(f"      • Invitation Key: {conn_invitation_key}...")
                        print(f"      • Their Label: {conn.get('their_label', 'N/A')}")
                        connection_found = True
                        break
                
                if not connection_found:
                    print(f"     ⚠️  Keine Connection mit matching invitation_msg_id gefunden")
                    print(f"     Gesuchte invitation_msg_id: {invitation_msg_id}")
            else:
                print(f"     ❌ Konnte Verifier Connections nicht abrufen")
        else:
            print("❌ Holder konnte Invitation nicht akzeptieren")
    else:
        print("❌ Fehler beim Erstellen der Invitation")

# ========================================
# 3. POST-VALIDATION: Connection-Status verifizieren
# ========================================
print("\n🔍 Post-Validation: Verifiziere Connection-Status...\n")

validation_success = False
verifier_state = None
holder_state = None

if conn_id_verifier_holder:
    # Prüfe Verifier-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    verifier_conn_detail = api_get(
        VERIFIER_ADMIN_URL,
        f"/connections/{conn_id_verifier_holder}"
    )
    
    if verifier_conn_detail:
        verifier_state = verifier_conn_detail.get("state")
        verifier_their_label = verifier_conn_detail.get("their_label")
        verifier_their_role = verifier_conn_detail.get("their_role")
        verifier_invitation_msg_id = verifier_conn_detail.get("invitation_msg_id")
        
        print(f"   📋 Verifier Connection {conn_id_verifier_holder}:")
        print(f"      • State: {verifier_state}")
        print(f"      • Their Label: {verifier_their_label}")
        print(f"      • Their Role: {verifier_their_role}")
        print(f"      • Invitation Msg ID: {verifier_invitation_msg_id}")

# Prüfe Holder-Seite (falls conn_id_holder_verifier bekannt)
if conn_id_holder_verifier:
    # Prüfe Holder-Seite mit spezifischem Endpoint: GET /connections/{conn_id}
    holder_conn_detail = api_get(
        HOLDER_ADMIN_URL,
        f"/connections/{conn_id_holder_verifier}"
    )
    
    if holder_conn_detail:
        holder_state = holder_conn_detail.get("state")
        holder_their_label = holder_conn_detail.get("their_label")
        holder_their_role = holder_conn_detail.get("their_role")
        holder_invitation_msg_id = holder_conn_detail.get("invitation_msg_id")
        
        print(f"\n   📋 Holder Connection {conn_id_holder_verifier}:")
        print(f"      • State: {holder_state}")
        print(f"      • Their Label: {holder_their_label}")
        print(f"      • Their Role: {holder_their_role}")
        print(f"      • Invitation Msg ID: {holder_invitation_msg_id}")
        
        # Verifiziere dass invitation_msg_id übereinstimmt
        if invitation_msg_id and verifier_invitation_msg_id == holder_invitation_msg_id:
            print(f"      ✅ Invitation Msg IDs stimmen überein!")

# Validierung
if verifier_state == "active" and holder_state == "active":
    validation_success = True
elif verifier_state == "active" and not conn_id_holder_verifier:
    # Bei existierender Connection nur Verifier-Seite bekannt
    validation_success = True

duration = time.time() - start_time
performance_metrics["connection_establishment"].append(duration)

# Finale Ausgabe
print(f"\n{'='*60}")
if validation_success:
    print(f"✅ CONNECTION ESTABLISHED (did:peer:2): Holder ↔ Verifier")
    print(f"   Verifier Connection ID: {conn_id_verifier_holder}")
    if conn_id_holder_verifier:
        print(f"   Holder Connection ID: {conn_id_holder_verifier}")
    print(f"   Verifier State: {verifier_state}")
    if holder_state:
        print(f"   Holder State: {holder_state}")
    if invitation_msg_id:
        print(f"   Invitation Msg ID: {invitation_msg_id}")
else:
    print(f"⚠️  CONNECTION STATUS UNCLEAR")
    print(f"   Verifier State: {verifier_state if verifier_state else 'N/A'}")
    print(f"   Holder State: {holder_state if holder_state else 'N/A'}")
    if conn_id_verifier_holder:
        print(f"   Verifier Connection ID: {conn_id_verifier_holder}")
    if conn_id_holder_verifier:
        print(f"   Holder Connection ID: {conn_id_holder_verifier}")

print(f"⏱️  Gesamtzeit: {duration:.3f}s")
print(f"{'='*60}")


In [ ]:
# Cell 15: Connection-Übersicht anzeigen

print("📊 Connection-Übersicht:\n")

connections_overview = []

# Issuer Connections
issuer_conns = api_get(ISSUER_ADMIN_URL, "/connections")
if issuer_conns and "results" in issuer_conns:
    for conn in issuer_conns["results"]:
        connections_overview.append({
            "Agent": "Issuer",
            "Partner": conn.get("their_label", "Unknown"),
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

# Holder Connections
holder_conns = api_get(HOLDER_ADMIN_URL, "/connections")
if holder_conns and "results" in holder_conns:
    for conn in holder_conns["results"]:
        partner = conn.get("their_label", "Unknown")
        connections_overview.append({
            "Agent": "Holder",
            "Partner": partner,
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

# Verifier Connections
verifier_conns = api_get(VERIFIER_ADMIN_URL, "/connections")
if verifier_conns and "results" in verifier_conns:
    for conn in verifier_conns["results"]:
        connections_overview.append({
            "Agent": "Verifier",
            "Partner": conn.get("their_label", "Unknown"),
            "Connection ID": conn["connection_id"],
            "State": conn["state"],
            "Invitation Msg ID": conn.get("invitation_msg_id", "N/A") if conn.get("invitation_msg_id") else "N/A",
            "Status": "✅" if conn["state"] == "active" else "⏳"
        })

df_connections = pd.DataFrame(connections_overview)
display(df_connections)

# Detaillierte Connection-Informationen
print("\n📋 Detaillierte Connection-Informationen:\n")

# Gruppiere nach invitation_msg_id um zusammengehörige Connections zu finden
all_connections = []
if issuer_conns and "results" in issuer_conns:
    for conn in issuer_conns["results"]:
        all_connections.append(("Issuer", conn))
if holder_conns and "results" in holder_conns:
    for conn in holder_conns["results"]:
        all_connections.append(("Holder", conn))
if verifier_conns and "results" in verifier_conns:
    for conn in verifier_conns["results"]:
        all_connections.append(("Verifier", conn))

# Gruppiere nach invitation_msg_id
from collections import defaultdict
connection_groups = defaultdict(list)
for agent, conn in all_connections:
    invitation_msg_id = conn.get("invitation_msg_id")
    if invitation_msg_id:
        connection_groups[invitation_msg_id].append((agent, conn))

# Zeige gruppierte Connections
for idx, (inv_msg_id, group) in enumerate(connection_groups.items(), 1):
    print(f"Connection Group {idx}:")
    print(f"   Invitation Msg ID: {inv_msg_id}")
    for agent, conn in group:
        print(f"   • {agent}: {conn['connection_id']} (State: {conn['state']})")
    print()


In [ ]:
# Cell 16 Wallet DID Übersicht anzeigen

print("📋 Wallet DID Übersicht - Alle DIDs:\n")

# Alle DIDs von allen Agenten abrufen
issuer_dids = api_get(ISSUER_ADMIN_URL, "/wallet/did")
holder_dids = api_get(HOLDER_ADMIN_URL, "/wallet/did")
verifier_dids = api_get(VERIFIER_ADMIN_URL, "/wallet/did")

# Zeige originale Responses
print("🔍 Originale /wallet/did Responses:\n")
if issuer_dids:
    pretty_print(issuer_dids, "Issuer Wallet DIDs")
if holder_dids:
    pretty_print(holder_dids, "Holder Wallet DIDs")
if verifier_dids:
    pretty_print(verifier_dids, "Verifier Wallet DIDs")

# Erstelle erweiterte Übersicht
print("\n📊 DID-Übersicht (Detailliert):\n")
ledger_overview = []

# Issuer DIDs
if issuer_dids and "results" in issuer_dids:
    for did_info in issuer_dids["results"]:
        posture = did_info.get("posture", "unknown")
        # Zeige alle DIDs (posted und wallet_only)
        ledger_overview.append({
            "Agent": "Issuer",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Holder DIDs
if holder_dids and "results" in holder_dids:
    for did_info in holder_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Holder",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Verifier DIDs
if verifier_dids and "results" in verifier_dids:
    for did_info in verifier_dids["results"]:
        posture = did_info.get("posture", "unknown")
        ledger_overview.append({
            "Agent": "Verifier",
            "DID": did_info["did"],
            "Verkey": did_info.get("verkey", "N/A"),
            "Posture": posture,
            "Method": did_info.get("method", "sov"),
            "Key Type": did_info.get("key_type", "N/A"),
            "On Ledger": "✅" if posture in ["public", "posted"] else "❌",
            "Endpoint": did_info.get("metadata", {}).get("endpoint", "N/A")
        })

# Als DataFrame anzeigen
if ledger_overview:
    df_ledger = pd.DataFrame(ledger_overview)
    display(df_ledger)
else:
    print("⚠️  Keine DIDs gefunden")

# Zusammenfassung
print("\n💡 Posture-Bedeutung:")
print("   • posted       = DID auf Ledger registriert und veröffentlicht")
print("   • public       = DID als Public DID gesetzt")
print("   • wallet_only  = DID nur im Wallet, nicht auf Ledger")
print("\n💡 Hinweis:")
print("   - Issuer & Verifier benötigen Public DIDs für Schema/CredDef")
print("   - Holder nutzt nur Peer-DIDs für sichere P2P-Verbindungen")
print("   - Alte DIDs (wallet_only) können aus vergangenen Runs stammen")

## Teil 5: Credential Issuance (Issuer → Holder)

Der Issuer stellt jetzt ein Credential an den Holder aus, das dessen Universitätsabschluss bescheinigt.

In [ ]:
# Cell 17: Credential Offer senden (Issuer → Holder)

print("📜 Credential Issuance: Issuer → Holder\n")

if conn_id_issuer_holder and cred_def_id:
    start_time = time.time()
    
    # Credential-Daten für Alice (Holder)
    credential_data = {
        "connection_id": conn_id_issuer_holder,
        "filter": {
            "indy": {
                "cred_def_id": cred_def_id
            }
        },
        "credential_preview": {
            "@type": "issue-credential/2.0/credential-preview",
            "attributes": [
                {"name": "name", "value": "Alice Smith"},
                {"name": "degree", "value": "Master of Computer Science"},
                {"name": "university", "value": "Technical University Munich"},
                {"name": "graduation_date", "value": "2024-07-15"},
                {"name": "age", "value": "26"},
                {"name": "gpa", "value": "3.8"}
            ]
        }
    }
    
    print("📋 Credential Preview:")
    for attr in credential_data["credential_preview"]["attributes"]:
        print(f"   • {attr['name']}: {attr['value']}")
    
    print("\n📤 Issuer sendet Credential Offer...")
    offer_response = api_post(
        ISSUER_ADMIN_URL,
        "/issue-credential-2.0/send-offer",
        credential_data
    )
    
    if offer_response and "cred_ex_id" in offer_response:
        cred_ex_id_issuer = offer_response["cred_ex_id"]
        initial_state = offer_response.get("state", "N/A")
        
        print(f"✅ Credential Offer gesendet")
        print(f"   Issuer Exchange ID: {cred_ex_id_issuer}")
        print(f"   Initial State: {initial_state}")
        
        # Warte auf Credential-Ausstellung (auto-accept auf Holder-Seite!)
        print("\n   Warte auf Credential-Ausstellung (auto-store auf Holder)...")
        time.sleep(5)
        
        # ========================================
        # STATUS-CHECK AUF BEIDEN SEITEN
        # ========================================
        print("\n🔍 Status-Check auf beiden Seiten:\n")
        
        # 1. Issuer-Seite Status
        issuer_state = None
        preserve_flag_active = False
        cred_status_issuer = api_get(
            ISSUER_ADMIN_URL,
            f"/issue-credential-2.0/records/{cred_ex_id_issuer}"
        )
        
        if cred_status_issuer:
            preserve_flag_active = True
            issuer_state = cred_status_issuer.get("cred_ex_record", {}).get("state", None)
            print(f"   📋 Issuer Exchange (via GET /records/{cred_ex_id_issuer}):")
            print(f"      • Exchange ID: {cred_ex_id_issuer}")
            print(f"      • State: {issuer_state}")
            print(f"      • Role: {cred_status_issuer.get("cred_ex_record", {}).get('role', 'N/A')}")
            print(f"      • Connection ID: {cred_status_issuer.get("cred_ex_record", {}).get('connection_id', 'N/A')}")
            print(f"      • Thread ID: {cred_status_issuer.get("cred_ex_record", {}).get('thread_id', 'N/A')}")
            print(f"      • Initiator: {cred_status_issuer.get("cred_ex_record", {}).get('initiator', 'N/A')}")
            print(f"      ✅ Record verfügbar (--preserve-exchange-records aktiv!)")
        else:
            print(f"   ⚠️  Issuer Exchange nicht abrufbar via spezifischer ID")
            print(f"      ❌ --preserve-exchange-records ist NICHT aktiv!")
            print(f"      🔧 Lösung: docker-compose restart issuer")
        
        # 2. Holder-Seite Status
        print("\n   📋 Holder Exchange(s):")
        holder_exchanges = api_get(HOLDER_ADMIN_URL, "/issue-credential-2.0/records")
        
        holder_cred_ex = None
        if holder_exchanges and "results" in holder_exchanges:
            for ex in holder_exchanges["results"]:
                ex_cred_def_id = None
                if "cred_offer" in ex:
                    cred_offer = ex.get("cred_offer", {})
                    ex_cred_def_id = cred_offer.get("cred_def_id")
                
                if ex_cred_def_id == cred_def_id:
                    holder_cred_ex = ex
                    print(f"      • Exchange ID: {ex['cred_ex_id']}")
                    print(f"      • State: {ex.get('state')}")
                    print(f"      • Role: {ex.get('role', 'N/A')}")
                    print(f"      • Cred Def ID: {ex_cred_def_id[:50]}...")
                    break
        
        # 3. Holder Credentials prüfen (finaler Erfolgs-Check!)
        print("\n   📋 Holder Credentials:")
        holder_creds = api_get(HOLDER_ADMIN_URL, "/credentials")
        
        credential_stored = False
        if holder_creds and "results" in holder_creds:
            for cred in holder_creds["results"]:
                if cred.get("cred_def_id") == cred_def_id:
                    credential_stored = True
                    print(f"      • Credential ID: {cred.get('referent', 'N/A')}")
                    print(f"      • Schema ID: {cred.get('schema_id', 'N/A')[:50]}...")
                    print(f"      • Cred Def ID: {cred.get('cred_def_id', 'N/A')[:50]}...")
                    
                    attrs = cred.get("attrs", {})
                    if attrs:
                        print(f"      • Attributes:")
                        for key, value in attrs.items():
                            print(f"         - {key}: {value}")
                    
                    credential_referent = cred.get("referent")
                    break
        
        duration = time.time() - start_time
        performance_metrics["credential_issuance"].append(duration)
        
        # ========================================
        # ISSUER CREDENTIAL REGISTRY (ENHANCED!)
        # ========================================
        print("\n" + "="*60)
        print("📋 ISSUER: Alle ausgestellten Credentials")
        print("="*60)
        
        all_issuer_records = api_get(ISSUER_ADMIN_URL, "/issue-credential-2.0/records")
        
        if all_issuer_records and "results" in all_issuer_records:
            results = all_issuer_records["results"]
            
            if len(results) > 0:
                print(f"\n✅ Issuer hat {len(results)} Credential Exchange Record(s)\n")
                
                for idx, record in enumerate(results, 1):
                    print(f"   Credential #{idx}:")
                    print(f"      • Exchange ID: {record['cred_ex_record']['cred_ex_id']}")
                    print(f"      • State: {record['cred_ex_record']['state']}")
                    print(f"      • Role: {record.get("cred_ex_record", {}).get('role', 'N/A')}")
                    print(f"      • Connection ID: {record.get("cred_ex_record", {}).get('connection_id', 'N/A')[:30]}...")
                    print(f"      • Thread ID: {record.get("cred_ex_record", {}).get('thread_id', 'N/A')}")
                    print(f"      • Initiator: {record.get("cred_ex_record", {}).get('initiator', 'N/A')}")
                    
                    # Cred Def ID aus cred_offer oder cred_issue
                    cred_def_from_record = None
                    if "cred_offer" in record:
                        cred_def_from_record = record.get("cred_offer", {}).get("cred_def_id")
                    elif "cred_issue" in record:
                        cred_def_from_record = record.get("cred_issue", {}).get("cred_def_id")
                    
                    if cred_def_from_record:
                        print(f"      • Cred Def ID: {cred_def_from_record[:50]}...")
                    
                    # Attributes aus Preview
                    if "cred_preview" in record:
                        preview = record.get("cred_preview", {})
                        attrs = preview.get("attributes", [])
                        if attrs:
                            print(f"      • Attributes:")
                            for attr in attrs[:3]:  # Erste 3 Attributes
                                print(f"         - {attr.get('name')}: {attr.get('value')}")
                            if len(attrs) > 3:
                                print(f"         ... und {len(attrs)-3} weitere")
                    
                    print(f"      • Created: {record.get("cred_ex_record", {}).get('created_at', 'N/A')}")
                    print(f"      • Updated: {record.get("cred_ex_record", {}).get('updated_at', 'N/A')}")
                    print()
            else:
                # ENHANCED: Better empty registry message
                print("\n⚠️  Registry ist LEER - keine Credential Exchange Records gefunden\n")
                print("   📌 WICHTIG: Nur Credentials die NACH dem Issuer-Neustart")
                print("              ausgestellt wurden, erscheinen hier!\n")
                
                if not preserve_flag_active:
                    print("   ❌ --preserve-exchange-records ist NICHT aktiv!")
                    print("   🔧 Lösung:")
                    print("      1. Prüfe docker-compose.yml (Zeile 42)")
                    print("      2. Führe aus: docker-compose restart issuer")
                    print("      3. Warte bis Issuer bereit ist (30-60 Sekunden)")
                    print("      4. Führe Cell 16 ERNEUT aus")
                else:
                    print("   ✅ --preserve-exchange-records ist aktiv")
                    print("   💡 Der gerade ausgestellte Credential sollte erscheinen")
                    print("      wenn du Cell 16 NOCHMAL ausführst!")
                print()
        else:
            print("\n❌ Konnte Registry nicht abrufen")
            print("   API-Fehler beim Zugriff auf /issue-credential-2.0/records")
        
        # ========================================
        # REVOCATION IDs SPEICHERN
        # ========================================
        if cred_status_issuer:
            rev_reg_id = cred_status_issuer.get("indy").get("rev_reg_id")
            cred_rev_id = cred_status_issuer.get("indy").get("cred_rev_id")
            
            if rev_reg_id and cred_rev_id:
                print("\n" + "="*60)
                print("🔑 REVOCATION-INFORMATIONEN")
                print("="*60)
                print(f"   • Revocation Registry ID: {rev_reg_id}")
                print(f"   • Credential Revocation ID: {cred_rev_id}")
                print("   💾 IDs gespeichert für Revocation-Workflow (Cell 29-34)")
                print("="*60 + "\n")
        
        # Finale Ausgabe
        print("="*60)
        if credential_stored:
            print(f"✅ CREDENTIAL ERFOLGREICH AUSGESTELLT UND GESPEICHERT")
            print(f"   Issuer State: {issuer_state if issuer_state else 'done'}")
            if holder_cred_ex:
                print(f"   Holder State: {holder_cred_ex.get('state')}")
            print(f"   Credential im Holder Wallet: ✅")
            
            if preserve_flag_active:
                print(f"   Issuer kann Credential dauerhaft einsehen: ✅")
            else:
                print(f"   Issuer kann Credential dauerhaft einsehen: ⚠️  (restart issuer)")
                print(f"   💡 Credential erscheint in Registry nach nächstem Issuer-Restart!")
        elif holder_cred_ex and holder_cred_ex.get('state') in ['credential-received', 'done']:
            print(f"✅ CREDENTIAL AUSGESTELLT")
            print(f"   Issuer State: {issuer_state if issuer_state else 'done'}")
            print(f"   Holder State: {holder_cred_ex.get('state')}")
            print(f"   ⚠️  Credential noch nicht im Wallet sichtbar (evtl. Indizierung)")
        else:
            print(f"⚠️  CREDENTIAL STATUS UNKLAR")
            print(f"   Issuer State: {issuer_state if issuer_state else 'N/A'}")
            if holder_cred_ex:
                print(f"   Holder State: {holder_cred_ex.get('state')}")
            print(f"   Credential im Wallet: ❌")
        
        print(f"⏱️  Gesamtzeit: {duration:.3f}s")
        print(f"{'='*60}")
        
        # Speichere Credential Referent für spätere Verwendung
        if credential_stored:
            for cred in holder_creds["results"]:
                if cred.get("cred_def_id") == cred_def_id:
                    credential_referent = cred.get("referent")
                    break
    else:
        print("❌ Fehler beim Senden des Credential Offers")
        if offer_response:
            print(f"   Response: {offer_response}")
else:
    print("⚠️  Übersprungen: Connection oder Cred Def fehlt")
    if not conn_id_issuer_holder:
        print("   • Connection Issuer→Holder fehlt (führe Cell 13 aus)")
    if not cred_def_id:
        print("   • Credential Definition fehlt (führe Cell 12 aus)")


In [ ]:
# Cell 18: Holder Credentials im Detail anzeigen

print("🎓 Holder Wallet - Gespeicherte Credentials:\n")

holder_creds = api_get(HOLDER_ADMIN_URL, "/credentials")

if holder_creds and "results" in holder_creds and len(holder_creds["results"]) > 0:
    for idx, cred in enumerate(holder_creds["results"], 1):
        print(f"\n{'='*60}")
        print(f"Credential #{idx}")
        print(f"{'='*60}")
        print(f"Referent: {cred.get('referent', 'N/A')}")
        print(f"Schema ID: {cred.get('schema_id', 'N/A')}")
        print(f"Cred Def ID: {cred.get('cred_def_id', 'N/A')}")
        
        # Credentials Revoked Abfrage gegen /credential/revoked/{credential_id ==> Referent}
        print(f"Revoked Status: {api_get(HOLDER_ADMIN_URL, f'/credential/revoked/{cred.get('referent', 'N/A')}').get('revoked', 'N/A')}")

        if "attrs" in cred:
            print("\nAttribute:")
            for attr_name, attr_value in cred["attrs"].items():
                print(f"  - {attr_name}: {attr_value}")
        
        
    
    # Als DataFrame anzeigen
    creds_list = []
    for cred in holder_creds["results"]:
        if "attrs" in cred:
            cred_info = {"Credential ID": cred.get("referent", "N/A")}
            cred_info.update(cred["attrs"])
            creds_list.append(cred_info)
    
    if creds_list:
        print("\n📊 Credential-Übersicht:")
        df_creds = pd.DataFrame(creds_list)
        display(df_creds)
else:
    print("⚠️  Holder hat noch keine Credentials")

## Teil 6: Proof Presentation (Holder → Verifier)

Der Verifier fordert jetzt einen Proof vom Holder an, um bestimmte Attribute zu verifizieren und Prädikate zu prüfen (z.B. Alter ≥ 18).

In [ ]:
# Cell 18: Proof Request senden (Verifier → Holder) - mit REVOCATION ALWAYS-ON
import time

# Aktueller Timestamp für Non-Revocation Intervall
current_timestamp = int(time.time())

print("📋 PROOF REQUEST mit ALWAYS-ON REVOCATION CHECK")
print(f"   Timestamp: {current_timestamp}")
print(f"   Non-Revoked Intervall: from=0, to={current_timestamp}\n")

# Proof Request Daten
proof_request_data = {
    "connection_id": conn_id_verifier_holder,
    "presentation_request": {
        "indy": {
            "name": "Proof of University Degree",
            "version": "1.0",
            "requested_attributes": {
                "attr1_referent": {
                    "name": "name",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}  # REVOCATION CHECK
                },
                "attr2_referent": {
                    "name": "degree",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}  # REVOCATION CHECK
                },
                "attr3_referent": {
                    "name": "university",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}  # REVOCATION CHECK
                },
                "attr4_referent": {
                    "name": "gpa",
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}  # REVOCATION CHECK
                }
            },
            "requested_predicates": {
                "predicate1_referent": {
                    "name": "age",
                    "p_type": ">=",
                    "p_value": 18,
                    "restrictions": [{"cred_def_id": cred_def_id}],
                    "non_revoked": {"from": 0, "to": current_timestamp}  # REVOCATION CHECK
                }
            },
            "non_revoked": {"to": current_timestamp}  # GLOBAL REVOCATION CHECK
        }
    }
}

# Sende Proof Request
proof_request_response = api_post(
    VERIFIER_ADMIN_URL,
    "/present-proof-2.0/send-request",
    proof_request_data
)

if proof_request_response is not None:
    pres_ex_id = proof_request_response.get("pres_ex_id")
    print(f"✅ Proof Request gesendet!")
    print(f"   Presentation Exchange ID: {pres_ex_id}")
    print(f"\n🔒 REVOCATION STATUS:")
    print(f"   • Non-Revoked Checks: AKTIV für alle Attribute & Prädikate")
    print(f"   • Credential muss zum Zeitpunkt {current_timestamp} NICHT REVOKED sein")
    print(f"   • Falls Credential revoked → Verification wird FEHLSCHLAGEN")
else:
    print("❌ Fehler beim Senden des Proof Requests")
    pres_ex_id = None


In [ ]:
# Cell 19: Holder - Proof Request empfangen und Credentials auswählen

print("📥 Holder: Proof Request verarbeiten...\n")

if pres_ex_id:
    # Holder prüft empfangene Proof Requests
    time.sleep(2)
    
    holder_pres_ex = api_get(HOLDER_ADMIN_URL, "/present-proof-2.0/records")
    
    if holder_pres_ex and "results" in holder_pres_ex:
        # Finde den aktuellen Presentation Exchange
        current_pres = None
        for pres in holder_pres_ex["results"]:
            if pres.get("state") == "request-received":
                current_pres = pres
                holder_pres_ex_id = pres["pres_ex_id"]
                break
        
        if current_pres:
            print(f"✅ Holder hat Proof Request empfangen")
            print(f"   Presentation Exchange ID: {holder_pres_ex_id}")
            
            # Credentials für die Presentation abrufen
            creds_for_pres = api_get(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}/credentials"
            )
            
            if creds_for_pres:
                print(f"\n📋 Verfügbare Credentials für Presentation:")
                
                # API kann entweder list oder dict zurückgeben
                total_creds = 0
                
                if isinstance(creds_for_pres, list):
                    # Liste von Credential Records
                    total_creds = len(creds_for_pres)
                    print(f"   Gefunden: {total_creds} Credential(s)")
                    
                    # Zeige Details der ersten paar Credentials
                    for idx, cred in enumerate(creds_for_pres[:3], 1):
                        print(f"\n   Credential #{idx}:")
                        if "cred_info" in cred:
                            cred_info = cred["cred_info"]
                            print(f"      • Referent: {cred_info.get('referent', 'N/A')}")
                            print(f"      • Schema ID: {cred_info.get('schema_id', 'N/A')}")
                            print(f"      • Cred Def ID: {cred_info.get('cred_def_id', 'N/A')}")
                            
                            # Zeige Attribute
                            attrs = cred_info.get("attrs", {})
                            if attrs:
                                print(f"      • Attributes:")
                                # Zeige erste 3 Attribute
                                shown = 0
                                for key, value in attrs.items():
                                    print(f"         - {key}: {value}")
                    
                elif isinstance(creds_for_pres, dict):
                    # Dictionary mit referent als key
                    for referent, creds in creds_for_pres.items():
                        if isinstance(creds, list):
                            total_creds += len(creds)
                            print(f"   {referent}: {len(creds)} Credential(s)")
                            
                            # Zeige Details des ersten Credentials
                            if len(creds) > 0:
                                cred = creds[0]
                                if "cred_info" in cred:
                                    cred_info = cred["cred_info"]
                                    print(f"      • Cred Def ID: {cred_info.get('cred_def_id', 'N/A')}")
                                    attrs = cred_info.get("attrs", {})
                                    if attrs:
                                        attr_count = len(attrs)
                                        print(f"      • {attr_count} Attribute(s) verfügbar")
                else:
                    print(f"   ⚠️  Unbekanntes Format: {type(creds_for_pres).__name__}")
                    total_creds = 0
                
                print(f"\n✅ Total: {total_creds} passende Credential(s) gefunden")
                
                if total_creds == 0:
                    print(f"\n⚠️  WARNUNG: Keine passenden Credentials gefunden!")
                    print(f"   • Stelle sicher dass Cell 16 erfolgreich war")
                    print(f"   • Prüfe ob Credential im Holder Wallet ist (Cell 16 Output)")
                    print(f"   • Schema/Cred Def müssen mit Proof Request übereinstimmen")
            else:
                print(f"\n⚠️  Keine Credentials für Presentation verfügbar")
                print(f"   API-Antwort war leer oder None")
        else:
            print("⚠️  Kein Proof Request im Status 'request-received'")
            print("\n   Verfügbare Presentation Exchanges:")
            for pres in holder_pres_ex["results"]:
                print(f"      • {pres['pres_ex_id']} State: {pres.get('state')}")
    else:
        print("⚠️  Keine Presentation Exchanges gefunden")
        print("   Führe Cell 18 aus um Proof Request zu senden")
else:
    print("⚠️  Übersprungen: Kein Presentation Exchange vorhanden")
    print("   Führe Cell 18 aus um Proof Request zu erstellen")


In [ ]:
# Cell 20: Holder - Presentation senden

print("📤 Holder: Presentation senden...\n")

if pres_ex_id and holder_pres_ex_id:
    start_time = time.time()
    
    # Hole Credentials nochmal
    creds_for_pres = api_get(
        HOLDER_ADMIN_URL,
        f"/present-proof-2.0/records/{holder_pres_ex_id}/credentials"
    )
    
    if creds_for_pres:
        print(f"📋 Credentials für Presentation gefunden")
        
        # Baue Presentation Request
        requested_credentials = {
            "indy": {
                "requested_attributes": {},
                "requested_predicates": {},
                "self_attested_attributes": {}
            }
        }
        
        # Handle verschiedene API-Response-Formate
        if isinstance(creds_for_pres, list):
            # Liste von Credential-Objekten
            print(f"   Format: Liste mit {len(creds_for_pres)} Credential(s)\n")
            
            # Jedes Credential-Objekt hat presentation_referents
            for cred_item in creds_for_pres:
                if "cred_info" in cred_item:
                    cred_id = cred_item["cred_info"].get("referent")
                    
                    # Hole presentation_referents (welche Felder dieses Cred erfüllen kann)
                    pres_referents = cred_item.get("presentation_referents", [])
                    
                    for ref in pres_referents:
                        # Prüfe ob es Attribute oder Predicate ist
                        if "attr" in ref or "attribute" in ref.lower():
                            requested_credentials["indy"]["requested_attributes"][ref] = {
                                "cred_id": cred_id,
                                "revealed": True
                            }
                            print(f"   ✓ Mapped attribute '{ref}' → Credential {cred_id}")
                        elif "pred" in ref or "predicate" in ref.lower():
                            requested_credentials["indy"]["requested_predicates"][ref] = {
                                "cred_id": cred_id
                            }
                            print(f"   ✓ Mapped predicate '{ref}' → Credential {cred_id}")
        
        elif isinstance(creds_for_pres, dict):
            # Dictionary mit referent als key (Legacy-Format)
            print(f"   Format: Dictionary mit {len(creds_for_pres)} Referent(s)\n")
            
            for referent, creds in creds_for_pres.items():
                if isinstance(creds, list) and len(creds) > 0:
                    first_cred = creds[0]
                    cred_id = first_cred.get("cred_info", {}).get("referent")
                    
                    if "attr" in referent:
                        requested_credentials["indy"]["requested_attributes"][referent] = {
                            "cred_id": cred_id,
                            "revealed": True
                        }
                        print(f"   ✓ Mapped attribute '{referent}' → Credential {cred_id[:20]}...")
                    elif "predicate" in referent:
                        requested_credentials["indy"]["requested_predicates"][referent] = {
                            "cred_id": cred_id
                        }
                        print(f"   ✓ Mapped predicate '{referent}' → Credential {cred_id[:20]}...")
        
        # Prüfe ob wir Credentials gemappt haben
        total_mapped = (
            len(requested_credentials["indy"]["requested_attributes"]) +
            len(requested_credentials["indy"]["requested_predicates"])
        )
        
        if total_mapped == 0:
            print("\n⚠️  WARNUNG: Keine Credentials gemappt!")
            print("   Versuche automatisches Mapping...\n")
            
            # Fallback: Verwende erstes verfügbares Credential für alle Requests
            # Hole Proof Request Details
            pres_ex_details = api_get(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}"
            )
            
            if pres_ex_details and "pres_request" in pres_ex_details:
                pres_request = pres_ex_details["pres_request"]
                
                # Hole erstes verfügbares Credential
                first_cred_id = None
                if isinstance(creds_for_pres, list) and len(creds_for_pres) > 0:
                    first_cred_id = creds_for_pres[0].get("cred_info", {}).get("referent")
                elif isinstance(creds_for_pres, dict):
                    for creds in creds_for_pres.values():
                        if isinstance(creds, list) and len(creds) > 0:
                            first_cred_id = creds[0].get("cred_info", {}).get("referent")
                            break
                
                if first_cred_id:
                    # Map alle requested attributes
                    if "requested_attributes" in pres_request:
                        for attr_ref in pres_request["requested_attributes"].keys():
                            requested_credentials["indy"]["requested_attributes"][attr_ref] = {
                                "cred_id": first_cred_id,
                                "revealed": True
                            }
                            print(f"   ✓ Auto-mapped attribute '{attr_ref}' → {first_cred_id[:20]}...")
                    
                    # Map alle requested predicates
                    if "requested_predicates" in pres_request:
                        for pred_ref in pres_request["requested_predicates"].keys():
                            requested_credentials["indy"]["requested_predicates"][pred_ref] = {
                                "cred_id": first_cred_id
                            }
                            print(f"   ✓ Auto-mapped predicate '{pred_ref}' → {first_cred_id[:20]}...")
                    
                    total_mapped = (
                        len(requested_credentials["indy"]["requested_attributes"]) +
                        len(requested_credentials["indy"]["requested_predicates"])
                    )
        
        print(f"\n📊 Presentation Mapping:")
        print(f"   • Requested Attributes: {len(requested_credentials['indy']['requested_attributes'])}")
        print(f"   • Requested Predicates: {len(requested_credentials['indy']['requested_predicates'])}")
        print(f"   • Total Mapped: {total_mapped}")
        
        if total_mapped > 0:
            # Sende Presentation
            print("\n📤 Sende Presentation...")
            presentation_response = api_post(
                HOLDER_ADMIN_URL,
                f"/present-proof-2.0/records/{holder_pres_ex_id}/send-presentation",
                requested_credentials
            )
            
            if presentation_response is not None:
                print(f"\n✅ Holder hat Presentation gesendet")
                print(f"   Presentation Exchange ID: {holder_pres_ex_id}")
                print(f"   State: {presentation_response.get('state', 'N/A')}")
                print(f"   Role: {presentation_response.get('role', 'N/A')}")
                
                # Warte auf Verification
                print("\n   Warte auf Verification durch Verifier...")
                time.sleep(3)
                
                # Prüfe finalen Status (kann 404 geben wenn abgeschlossen!)
                final_status = api_get(
                    HOLDER_ADMIN_URL,
                    f"/present-proof-2.0/records/{holder_pres_ex_id}"
                )
                
                if final_status:
                    # Record noch vorhanden (--preserve-exchange-records aktiv)
                    final_state = final_status.get("state")
                    print(f"\n   📋 Finaler Holder Status: {final_state}")
                    
                    if final_state in ["done", "presentation-sent"]:
                        print(f"   ✅ Presentation erfolgreich abgeschlossen!")
                        print(f"   ✅ Record bleibt verfügbar (--preserve-exchange-records aktiv)")
                    elif final_state == "abandoned":
                        print(f"   ⚠️  Presentation wurde abgebrochen")
                    else:
                        print(f"   ⏳ Presentation in Bearbeitung...")
                else:
                    # 404 Error = Record wurde gelöscht = erfolgreich abgeschlossen!
                    print(f"\n   📋 Finaler Holder Status: completed/archived (404)")
                    print(f"   ✅ Presentation erfolgreich abgeschlossen!")
                    print(f"   ℹ️  Record wurde nach Completion gelöscht (--preserve-exchange-records nicht aktiv)")
                    print(f"\n   💡 Hinweis: Für dauerhafte Records, starte Holder neu:")
                    print(f"      docker-compose restart holder")
                
                duration = time.time() - start_time
                
                # Initialize key if not exists
                if "presentation" not in performance_metrics:
                    performance_metrics["presentation"] = []
                
                performance_metrics["presentation"].append(duration)
                print(f"\n⏱️  Presentation Zeit: {duration:.3f}s")
            else:
                print("\n❌ Fehler beim Senden der Presentation")
                print("   API-Response war leer oder fehlerhaft")
        else:
            print("\n❌ Kann Presentation nicht senden - keine Credentials gemappt")
            print("   • Prüfe ob Credentials die Proof Request Anforderungen erfüllen")
            print("   • Prüfe Schema und Cred Def IDs")
    else:
        print("⚠️  Keine Credentials für Presentation gefunden")
        print("   • Führe Cell 16 aus um Credential auszustellen")
        print("   • Prüfe ob Credential im Holder Wallet ist")
else:
    print("⚠️  Übersprungen: Presentation Exchange nicht vorhanden")
    if not pres_ex_id:
        print("   • Verifier Presentation Exchange ID fehlt (führe Cell 18 aus)")
    if not holder_pres_ex_id:
        print("   • Holder Presentation Exchange ID fehlt (führe Cell 19 aus)")


In [ ]:
# Cell 21: Verifier - Presentation verifizieren (mit Revocation Detection)

print("🔍 Verifier: Presentation verifizieren...\n")

if pres_ex_id:
    # ========================================
    # WAIT FOR PRESENTATION (mit Timeout)
    # ========================================
    print("⏳ Warte auf Presentation vom Holder...")

    max_wait = 15  # Sekunden
    wait_interval = 2
    waited = 0

    while waited < max_wait:
        verifier_pres_status = api_get(
            VERIFIER_ADMIN_URL,
            f"/present-proof-2.0/records/{pres_ex_id}"
        )

        state = verifier_pres_status.get("state", "N/A")

        if state in ["done", "abandoned", "presentation-received"]:
            break

        time.sleep(wait_interval)
        waited += wait_interval
        print(f"   ... warte ({waited}s, State: {state})")

    duration = time.time() - start_time
    performance_metrics["proof_presentation"].append(duration)

    # ========================================
    # ANALYZE VERIFICATION RESULT
    # ========================================
    if verifier_pres_status:
        state = verifier_pres_status.get("state", "N/A")
        verified = verifier_pres_status.get("verified", "N/A")
        error_msg = verifier_pres_status.get("error_msg")

        print(f"\n📊 Verification Status:")
        print(f"   • State: {state}")
        print(f"   • Verified: {verified}")
        print(f"   • Zeit: {duration:.3f}s")

        # ========================================
        # REVOCATION DETECTION
        # ========================================
        is_revoked = False
        revocation_error = None

        # Check 1: Error message enthält "revoked"
        if error_msg and "revok" in error_msg.lower():
            is_revoked = True
            revocation_error = error_msg

        # Check 2: State ist "abandoned" + verified ist False
        if state == "abandoned" and (verified == "false" or verified is False):
            is_revoked = True

        # Check 3: Verified False aber State Done (revocation after issuance)
        if (verified == "false" or verified is False) and state == "done":
            # Prüfe ob non_revoked war im Request
            pres_request = verifier_pres_status.get("pres_request", {})
            if "non_revoked" in str(pres_request):
                is_revoked = True

        # ========================================
        # OUTPUT BASED ON STATUS
        # ========================================

        if is_revoked:
            # CREDENTIAL WAS REVOKED
            print("\n" + "="*60)
            print("❌ CREDENTIAL IST REVOKED!")
            print("="*60)
            print("🚫 Das Credential wurde vom Issuer widerrufen")
            print()
            print("Details:")
            print("   • Der Holder kann keinen gültigen Proof erstellen")
            print("   • Non-Revocation Check ist fehlgeschlagen")
            if revocation_error:
                print(f"   • Error: {revocation_error[:100]}...")
            print()
            print("💡 Mögliche Gründe:")
            print("   • Credential wurde in Cell 23 revoked")
            print("   • Revocation wurde in Cell 24 published")
            print("   • Ledger hat Revocation bestätigt")
            print("="*60)

        elif verified == "true" or verified is True:
            # VERIFICATION SUCCESSFUL
            print("\n" + "="*60)
            print("✅ PROOF VERIFICATION ERFOLGREICH!")
            print("="*60)

            # Präsentierte Daten anzeigen
            if "by_format" in verifier_pres_status and "pres" in verifier_pres_status["by_format"]:
                pres_data = verifier_pres_status["by_format"]["pres"]

                if "indy" in pres_data:
                    indy_pres = pres_data["indy"]

                    print("\n📋 Präsentierte Attribute:")
                    if "requested_proof" in indy_pres:
                        revealed_attrs = indy_pres["requested_proof"].get("revealed_attrs", {})
                        for referent, attr_data in revealed_attrs.items():
                            attr_name = referent.split("_")[1] if "_" in referent else referent
                            print(f"   • {attr_name}: {attr_data.get('raw', 'N/A')}")

                    print("\n✅ Verifizierte Prädikate:")
                    if "requested_proof" in indy_pres:
                        predicates = indy_pres["requested_proof"].get("predicates", {})
                        for referent in predicates.keys():
                            pred_name = referent.split("_")[1] if "_" in referent else referent
                            print(f"   • {pred_name}: ✅ Erfüllt")

                    # Non-Revocation Status
                    print("\n🔓 Revocation Status:")
                    if "identifiers" in indy_pres:
                        for identifier in indy_pres["identifiers"]:
                            rev_reg_id = identifier.get("rev_reg_id")
                            timestamp = identifier.get("timestamp")
                            if rev_reg_id:
                                print(f"   • Credential ist NICHT revoked")
                                print(f"   • Geprüft zum Zeitpunkt: {timestamp}")
                                print(f"   • Registry: {rev_reg_id}")
                            else:
                                print(f"   • Kein Revocation-Check (Registry nicht verwendet)")

            print("\n" + "="*60)
            print("Der Verifier hat erfolgreich verifiziert:")
            print("  ✅ Alle Attribute sind korrekt")
            print("  ✅ Alle Prädikate sind erfüllt")
            print("  ✅ Credential ist NICHT revoked")
            print("  ✅ Zero-Knowledge Proofs sind gültig")
            print("="*60)

        elif state == "request-sent":
            # STILL WAITING
            print("\n⏰ TIMEOUT: Holder hat nicht rechtzeitig geantwortet")
            print(f"   State ist noch: {state}")
            print("\n💡 Mögliche Ursachen:")
            print("   • Holder Agent ist offline")
            print("   • Connection ist broken")
            print("   • Holder hat Credential nicht im Wallet")
            print("   • Holder kann Non-Revocation Proof nicht erstellen (revoked?)")

        else:
            # OTHER ERROR
            print("\n⚠️  Verification fehlgeschlagen")
            print(f"   State: {state}")
            print(f"   Verified: {verified}")
            if error_msg:
                print(f"   Error: {error_msg}")

            # Try to determine if it's revocation-related
            if error_msg and any(word in error_msg.lower() for word in ["revok", "tail", "registry"]):
                print("\n💡 Hinweis: Fehler könnte mit Revocation zusammenhängen")
                print("   • Prüfe ob Credential revoked wurde (Cell 23-24)")
                print("   • Prüfe Tails-Server Logs: docker logs tails-server")
    else:
        print("❌ Fehler beim Abrufen des Verification-Status")

else:
    print("⚠️  Übersprungen: Kein Presentation Exchange vorhanden")
    print("   • Führe Cell 19 aus um Proof Request zu senden")

## Teil 7: Revocation

Jetzt wird revocated.

In [ ]:
# Cell 22: Revocation Registries anzeigen

print("="*60)
print("📋 ISSUER: Revocation Registries")
print("="*60 + "\n")

registries = api_get(
    ISSUER_ADMIN_URL,
    "/revocation/registries/created?state=active"
)

if registries and "rev_reg_ids" in registries:
    reg_ids = registries["rev_reg_ids"]
    print(f"✅ Aktive Revocation Registries: {len(reg_ids)}\n")
    
    for idx, reg_id in enumerate(reg_ids, 1):
        print(f"Registry #{idx}: {reg_id}")
        
        # Registry Details abrufen
        reg_info = api_get(ISSUER_ADMIN_URL, f"/revocation/registry/{reg_id}").get('result', 'N/A')
        if reg_info:
            print(f"   • State: {reg_info.get('state')}")
            print(f"   • Max Credentials: {reg_info.get('max_cred_num')}")
            print(f"   • Issuer DID: {reg_info.get('issuer_did', 'N/A')}")
            print(f"   • Tails Hash: {reg_info.get('tails_hash', 'N/A')}")
            print(f"   • Tails Location: {reg_info.get('tails_local_path', 'N/A')}")
            print(f"   • Created: {reg_info.get('created_at', 'N/A')}\n")
else:
    print("⚠️  Keine aktiven Revocation Registries gefunden")
    print("   • Stelle sicher dass Cell 12 mit support_revocation: True läuft")
    print("   • Führe Cell 16 aus um Credential mit Revocation auszustellen")

print("="*60)


In [ ]:
# Cell 23: Credential REVOKEN (staged)

print("="*60)
print("🚫 Credential REVOKEN (staged)")
print("="*60 + "\n")

# Verwende gespeicherte Werte aus Cell 16
if 'rev_reg_id' in locals() and 'cred_rev_id' in locals():
    print(f"📋 Zu revokendes Credential:")
    print(f"   • Rev Reg ID: {rev_reg_id}")
    print(f"   • Cred Rev ID: {cred_rev_id}\n")
    
    revoke_request = {
        "rev_reg_id": rev_reg_id,
        "cred_rev_id": cred_rev_id,
        "notify": False,  # Kein Notification-Webhook
        "publish": False,  # Staging only, publishe später in Cell 31
        "comment": "Revoked for testing purposes"
    }
    
    print("📤 Sende Revocation Request (staging)...")
    start_time = time.time()
    
    revoke_response = api_post(
        ISSUER_ADMIN_URL,
        "/revocation/revoke",
        revoke_request
    )
    
    duration = time.time() - start_time
    
    if revoke_response is not None:
        print(f"\n✅ Credential erfolgreich REVOKED (staged)")
        print(f"   • Status: Pending (noch nicht auf Ledger)")
        print(f"   • Zeit: {duration:.3f}s")
        print(f"\n💡 Nächster Schritt: Führe Cell 31 aus um auf Ledger zu publishen!")
    else:
        print(f"\n❌ Revocation fehlgeschlagen")
        print(f"   Prüfe ob die IDs korrekt sind")
else:
    print("⚠️  Keine Revocation IDs gefunden!")
    print("   • Führe Cell 16 aus um Credential auszustellen")
    print("   • Die IDs werden automatisch in rev_reg_id und cred_rev_id gespeichert")

print("\n" + "="*60)


In [ ]:
# Cell 24: Revocations auf Ledger PUBLISHEN

print("="*60)
print("📤 PUBLISH Revocations auf Ledger")
print("="*60 + "\n")

print("Publishing alle pending Revocations auf Ledger...")
start_time = time.time()

publish_response = api_post(
    ISSUER_ADMIN_URL,
    "/revocation/publish-revocations",
    {}
)

duration = time.time() - start_time

if publish_response and "rrid2crid" in publish_response:
    revoked_regs = publish_response["rrid2crid"]
    total_revoked = sum(len(creds) for creds in revoked_regs.values())
    
    print(f"\n✅ ERFOLGREICH auf Ledger published:")
    print(f"   • Revocation Registries: {len(revoked_regs)}")
    print(f"   • Total revoked Credentials: {total_revoked}")
    print(f"   • Publish Zeit: {duration:.3f}s\n")
    
    for reg_id, cred_ids in revoked_regs.items():
        print(f"   📋 Registry: {reg_id}")
        print(f"      Revoked Credential IDs: {cred_ids}")
        print()
    
    print("="*60)
    print("✅ Revocations sind jetzt auf dem VON Ledger!")
    print("   💡 Holder kann nun KEINEN Non-Revocation-Proof mehr erstellen")
    print("="*60)
elif publish_response:
    print(f"\n⚠️  Keine pending Revocations zum Publishen")
    print(f"   Response: {publish_response}")
else:
    print(f"\n❌ Publish fehlgeschlagen")

print()


In [ ]:
# Cell 25: Proof Request MIT Revocation-Check

print("="*60)
print("📜 Proof Request MIT Revocation-Check")
print("="*60 + "\n")

if conn_id_verifier_holder and cred_def_id:
    current_timestamp = int(time.time())
    
    # Proof Request mit non_revoked Interval
    proof_request = {
        "comment": "Proof Request with Non-Revocation Check",
        "presentation_request": {
            "indy": {
                "name": "Proof of Education (Non-Revoked)",
                "version": "1.0",
                "requested_attributes": {
                    "attr1_referent": {
                        "name": "name",
                        "restrictions": [{"cred_def_id": cred_def_id}],
                        "non_revoked": {
                            "from": 0,
                            "to": current_timestamp
                        }
                    },
                    "attr2_referent": {
                        "name": "degree",
                        "restrictions": [{"cred_def_id": cred_def_id}],
                        "non_revoked": {
                            "from": 0,
                            "to": current_timestamp
                        }
                    }
                },
                "requested_predicates": {},
                "non_revoked": {
                    "from": 0,
                    "to": current_timestamp
                }
            }
        },
        "connection_id": conn_id_verifier_holder
    }
    
    print(f"📋 Non-Revocation Interval:")
    print(f"   • From: 0 (Genesis)")
    print(f"   • To: {current_timestamp} (jetzt)\n")
    print("💡 Holder muss beweisen: Credential ist NICHT revoked!\n")
    
    print("📤 Sende Proof Request...")
    start_time = time.time()
    
    pres_ex_response = api_post(
        VERIFIER_ADMIN_URL,
        "/present-proof-2.0/send-request",
        proof_request
    )
    
    duration = time.time() - start_time
    
    if pres_ex_response is not None:
        verifier_pres_ex_id = pres_ex_response["pres_ex_id"]
        print(f"\n✅ Proof Request gesendet")
        print(f"   • Pres Ex ID: {verifier_pres_ex_id}")
        print(f"   • State: {pres_ex_response.get('state')}")
        print(f"   • Zeit: {duration:.3f}s")
        print(f"\n   ⏳ Warte auf Holder Response (auto-respond aktiv)...")
    else:
        print(f"\n❌ Fehler beim Senden des Proof Request")
else:
    print("⚠️  Übersprungen: Connection oder Cred Def fehlt")
    if not conn_id_verifier_holder:
        print("   • Connection Verifier→Holder fehlt (führe Cell 14 aus)")
    if not cred_def_id:
        print("   • Credential Definition fehlt (führe Cell 12 aus)")

print("\n" + "="*60)


In [ ]:
# Cell 26: Presentation Verification (mit Revocation-Check)

print("\n⏳ Warte auf Presentation und Verification...\n")
time.sleep(5)

print("="*60)
print("🔍 VERIFICATION Result (mit Revocation-Check)")
print("="*60 + "\n")

if 'verifier_pres_ex_id' in locals():
    # Hole Verifier Presentation Exchange
    verifier_pres = api_get(
        VERIFIER_ADMIN_URL,
        f"/present-proof-2.0/records/{verifier_pres_ex_id}"
    )
    
    if verifier_pres:
        state = verifier_pres.get("state")
        verified = verifier_pres.get("verified")
        
        print(f"📋 Presentation Exchange:")
        print(f"   • Pres Ex ID: {verifier_pres_ex_id}")
        print(f"   • State: {state}")
        print(f"   • Verified: {verified}")
        print(f"   • Role: {verifier_pres.get('role')}\n")
        
        if verified == "true":
            print("✅ VERIFICATION SUCCESSFUL!")
            print("   ✓ Credential ist NICHT revoked")
            print("   ✓ Non-Revocation Proof wurde erfolgreich verifiziert")
            print("   ✓ Holder hat Zugriff auf Tails-File vom Tails-Server erhalten")
            print("   ✓ Zero-Knowledge Proof: Credential Status ist gültig")
        elif verified == "false":
            print("❌ VERIFICATION FAILED!")
            print("   ✗ Credential wurde REVOKED")
            print("   ✗ Non-Revocation Proof konnte NICHT erstellt werden")
            print("   ✗ Revocation ist auf Ledger published")
            print("   💡 Credential ist ungültig - Holder kann keinen Beweis erbringen!")
        else:
            print(f"⏳ Status: {state}")
            print("   Warte noch auf Verification...")
        
        # Zeige Presentation Details wenn vorhanden
        if "by_format" in verifier_pres:
            print(f"\n   📋 Presentation Format:")
            for fmt, data in verifier_pres["by_format"].items():
                print(f"      • Format: {fmt}")
    else:
        print("❌ Konnte Presentation Exchange nicht abrufen")
else:
    print("⚠️  Keine Presentation Exchange ID gefunden")
    print("   Führe Cell 32 aus um Proof Request zu senden")

print("\n" + "="*60)


In [ ]:
# Cell 27: Revocation Registry ROTATION (Optional)

print("="*60)
print("🔄 Revocation Registry ROTATION")
print("="*60 + "\n")

print("💡 Registry Rotation ist nötig wenn:")
print("   • Registry voll ist (100 Credentials mit TAILS_FILE_COUNT=100)")
print("   • Neue Registry für weitere Credentials benötigt wird\n")

if 'cred_def_id' in locals() and cred_def_id:
    print(f"Rotiere Registry für Cred Def: {cred_def_id[:50]}...\n")
    
    start_time = time.time()
    
    rotate_response = api_post(
        ISSUER_ADMIN_URL,
        f"/revocation/active-registry/{cred_def_id}/rotate",
        {}
    )
    
    duration = time.time() - start_time
    
    if rotate_response and "rev_reg_ids" in rotate_response:
        decom = rotate_response["rev_reg_ids"]
        print(f"✅ Registry erfolgreich rotiert!")
        print(f"   • Decommissioned Registries: {len(decom)}")
        for reg_id in decom:
            print(f"      - {reg_id[:50]}...")
        print(f"   • Zeit: {duration:.3f}s")
        print(f"\n   💡 Neue aktive Registry wurde erstellt")
        print(f"   💡 Weitere Credentials können jetzt ausgestellt werden")
    else:
        print(f"⚠️  Rotation nicht möglich")
        print(f"   • Möglicherweise ist die Registry noch nicht voll")
        print(f"   • Oder es gibt keine aktive Registry für diese Cred Def")
        if rotate_response:
            print(f"   Response: {rotate_response}")
else:
    print("⚠️  Keine Credential Definition ID gefunden")
    print("   Führe Cell 12 aus um Cred Def zu erstellen")

print("\n" + "="*60)


## Teil 8: Performance-Analyse & Visualisierungen

Jetzt analysieren wir die Performance-Metriken des gesamten SSI-Workflows und visualisieren die Ergebnisse.

In [ ]:
# Cell 28: Performance-Metriken Zusammenfassung

print("📊 Performance-Metriken Zusammenfassung\n")
print("="*70)

# Berechne Statistiken für jede Operation
performance_summary = []

for operation, times in performance_metrics.items():
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        total_time = sum(times)
        count = len(times)
        
        performance_summary.append({
            "Operation": operation.replace("_", " ").title(),
            "Count": count,
            "Avg (s)": f"{avg_time:.3f}",
            "Min (s)": f"{min_time:.3f}",
            "Max (s)": f"{max_time:.3f}",
            "Total (s)": f"{total_time:.3f}"
        })
        
        print(f"\n{operation.replace('_', ' ').upper()}")
        print(f"  Anzahl:      {count}")
        print(f"  Durchschnitt: {avg_time:.3f}s")
        print(f"  Minimum:     {min_time:.3f}s")
        print(f"  Maximum:     {max_time:.3f}s")
        print(f"  Gesamt:      {total_time:.3f}s")

# Als DataFrame anzeigen
if performance_summary:
    print("\n" + "="*70)
    df_performance = pd.DataFrame(performance_summary)
    display(df_performance)
    
    # Gesamtzeit berechnen
    total_workflow_time = sum(sum(times) for times in performance_metrics.values() if times)
    print(f"\n⏱️  GESAMT-WORKFLOW-ZEIT: {total_workflow_time:.3f}s")
else:
    print("\n⚠️  Keine Performance-Daten verfügbar")

In [ ]:
# Cell 29: Visualisierung - Performance Balkendiagramm

import matplotlib.pyplot as plt
import numpy as np

print("📊 Performance Visualisierung - Balkendiagramm\n")

# Bereite Daten vor
operations = []
avg_times = []

for operation, times in performance_metrics.items():
    if times:
        operations.append(operation.replace("_", " ").title())
        avg_times.append(sum(times) / len(times))

if operations:
    # Erstelle Balkendiagramm
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = [ISSUER_COLOR, ISSUER_COLOR, ISSUER_COLOR, ISSUER_COLOR, 
              HOLDER_COLOR, ISSUER_COLOR, VERIFIER_COLOR][:len(operations)]
    
    bars = ax.bar(operations, avg_times, color=colors, alpha=0.7, edgecolor='black')
    
    # Beschriftungen
    ax.set_xlabel('Operation', fontsize=12, fontweight='bold')
    ax.set_ylabel('Durchschnittliche Zeit (Sekunden)', fontsize=12, fontweight='bold')
    ax.set_title('SSI Workflow - Performance pro Operation', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Werte auf Balken anzeigen
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}s',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Rotiere X-Achsen-Labels für bessere Lesbarkeit
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print("✅ Balkendiagramm erstellt")
else:
    print("⚠️  Keine Daten zum Visualisieren")

In [ ]:
# Cell 30: Visualisierung - Kreisdiagramm (Zeitanteil pro Operation)

print("📊 Performance Visualisierung - Kreisdiagramm\n")

# Bereite Daten vor
operations = []
total_times = []

for operation, times in performance_metrics.items():
    if times:
        operations.append(operation.replace("_", " ").title())
        total_times.append(sum(times))

if operations and sum(total_times) > 0:
    # Erstelle Kreisdiagramm
    fig, ax = plt.subplots(figsize=(10, 8))
    
    colors_pie = [ISSUER_COLOR, ISSUER_COLOR, ISSUER_COLOR, ISSUER_COLOR, 
                  HOLDER_COLOR, ISSUER_COLOR, VERIFIER_COLOR][:len(operations)]
    
    wedges, texts, autotexts = ax.pie(
        total_times,
        labels=operations,
        colors=colors_pie,
        autopct='%1.1f%%',
        startangle=90,
        textprops={'fontsize': 10, 'fontweight': 'bold'}
    )
    
    # Titel
    ax.set_title('SSI Workflow - Zeitverteilung pro Operation', 
                 fontsize=14, fontweight='bold', pad=20)
    
    # Legende mit absoluten Zeiten
    legend_labels = [f"{op}: {time:.3f}s" for op, time in zip(operations, total_times)]
    ax.legend(legend_labels, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Kreisdiagramm erstellt")
else:
    print("⚠️  Keine Daten zum Visualisieren")

In [ ]:
# Cell 31: Workflow-Sequenzdiagramm Visualisierung

print("📊 SSI Workflow - Sequenzdiagramm\n")

display(Markdown("""
## 🔄 Kompletter SSI Workflow - Sequenz

```
┌─────────┐          ┌─────────┐          ┌─────────┐          ┌─────────┐
│ Issuer  │          │ Holder  │          │Verifier │          │ Ledger  │
└────┬────┘          └────┬────┘          └────┬────┘          └────┬────┘
     │                    │                    │                    │
     │ 1. DID erstellen   │                    │                    │
     ├────────────────────┼────────────────────┼───────────────────▶│
     │◀───────────────────┼────────────────────┼────────────────────┤
     │                    │                    │                    │
     │ 2. Schema erstellen│                    │                    │
     ├────────────────────┼────────────────────┼───────────────────▶│
     │◀───────────────────┼────────────────────┼────────────────────┤
     │                    │                    │                    │
     │ 3. Cred Def erstellen                   │                    │
     ├────────────────────┼────────────────────┼───────────────────▶│
     │◀───────────────────┼────────────────────┼────────────────────┤
     │                    │                    │                    │
     │ 4. Connection      │                    │                    │
     ├───────────────────▶│                    │                    │
     │◀───────────────────┤                    │                    │
     │                    │                    │                    │
     │ 5. Credential Offer│                    │                    │
     ├───────────────────▶│                    │                    │
     │                    │                    │                    │
     │ 6. Credential Request                   │                    │
     │◀───────────────────┤                    │                    │
     │                    │                    │                    │
     │ 7. Issue Credential│                    │                    │
     ├───────────────────▶│                    │                    │
     │                    │                    │                    │
     │                    │ 8. Connection      │                    │
     │                    ├───────────────────▶│                    │
     │                    │◀───────────────────┤                    │
     │                    │                    │                    │
     │                    │ 9. Proof Request   │                    │
     │                    │◀───────────────────┤                    │
     │                    │                    │                    │
     │                    │ 10. Presentation   │                    │
     │                    ├───────────────────▶│                    │
     │                    │                    │                    │
     │                    │                    │ 11. Verify         │
     │                    │                    ├───────────────────▶│
     │                    │                    │◀───────────────────┤
     │                    │                    │                    │
     │                    │ 12. Verification OK│                    │
     │                    │◀───────────────────┤                    │
     │                    │                    │                    │
```

**Workflow-Phasen:**
1. **Setup** (Schritte 1-3): Ledger-Vorbereitung durch Issuer
2. **Issuance** (Schritte 4-7): Credential-Ausstellung an Holder
3. **Verification** (Schritte 8-12): Proof-Präsentation und -Verifikation
"""))

In [ ]:
# Cell 32: Performance Bottleneck-Analyse

print("🔍 Performance Bottleneck-Analyse\n")
print("="*70)

# Finde die langsamsten Operationen
operations_with_times = []
for operation, times in performance_metrics.items():
    if times:
        avg_time = sum(times) / len(times)
        operations_with_times.append((operation, avg_time, sum(times)))

# Sortiere nach durchschnittlicher Zeit (absteigend)
operations_with_times.sort(key=lambda x: x[1], reverse=True)

print("\n📊 Top 3 Langsamste Operationen (nach Durchschnittszeit):\n")
for i, (operation, avg_time, total_time) in enumerate(operations_with_times[:3], 1):
    print(f"{i}. {operation.replace('_', ' ').title()}")
    print(f"   Durchschnitt: {avg_time:.3f}s")
    print(f"   Gesamt:       {total_time:.3f}s")
    print()

# Berechne Anteile
if operations_with_times:
    total_time_all = sum(x[2] for x in operations_with_times)
    
    print("="*70)
    print("\n💡 Performance-Empfehlungen:\n")
    
    # Analysiere die Bottlenecks
    if operations_with_times:
        slowest_op, slowest_avg, slowest_total = operations_with_times[0]
        percentage = (slowest_total / total_time_all) * 100
        
        print(f"1. Hauptbottleneck: {slowest_op.replace('_', ' ').title()}")
        print(f"   Macht {percentage:.1f}% der Gesamtzeit aus")
        
        # Gebe spezifische Empfehlungen basierend auf Operation
        if "connection" in slowest_op:
            print("   → Optimierung: Connection-Pooling oder persistente Verbindungen")
        elif "credential_issuance" in slowest_op:
            print("   → Optimierung: Batch-Credential-Issuance implementieren")
        elif "proof_presentation" in slowest_op:
            print("   → Optimierung: Cached Credential-Verification nutzen")
        elif "schema_creation" in slowest_op or "cred_def" in slowest_op:
            print("   → Optimierung: Schema/CredDef nur einmalig erstellen, wiederverwenden")
        elif "ledger" in slowest_op:
            print("   → Optimierung: Ledger-Schreibzugriffe minimieren, Batch-Writes")
    
    print("\n2. Allgemeine Optimierungen:")
    print("   → Parallele Verarbeitung für unabhängige Operationen")
    print("   → Caching von häufig genutzten Ledger-Daten")
    print("   → Asynchrone API-Calls wo möglich")
    print("   → Connection-Reuse zwischen Operationen")

## Teil 9: Zusammenfassung & Abschluss

Finaler Überblick über den kompletten SSI-Workflow mit allen Ergebnissen.

In [ ]:
# Cell 33: Workflow-Zusammenfassung

print("="*70)
print("🎉 SSI WORKFLOW ERFOLGREICH ABGESCHLOSSEN!")
print("="*70)

# Zähle erfolgreiche Operationen
summary = {
    "Agenten": {
        "Issuer": "✅ Aktiv",
        "Holder": "✅ Aktiv",
        "Verifier": "✅ Aktiv"
    },
    "Ledger-Operationen": {
        "DIDs registriert": 2,  # Issuer + Verifier
        "Schemas erstellt": 1,
        "Credential Definitions": 1
    },
    "Connections": {
        "Issuer ↔ Holder": "✅ Etabliert",
        "Holder ↔ Verifier": "✅ Etabliert"
    },
    "Credentials": {
        "Ausgestellt": 1,
        "Gespeichert (Holder)": len(api_get(HOLDER_ADMIN_URL, "/credentials").get("results", [])) if api_get(HOLDER_ADMIN_URL, "/credentials") else 0
    },
    "Proof Presentations": {
        "Angefordert": 1,
        "Präsentiert": 1,
        "Verifiziert": "✅" if verifier_pres_status and verifier_pres_status.get("verified") else "⏳"
    }
}

print("\n📊 WORKFLOW-STATISTIK:\n")

for category, items in summary.items():
    print(f"\n{category}:")
    if isinstance(items, dict):
        for key, value in items.items():
            print(f"  • {key}: {value}")
    else:
        print(f"  • {items}")

# Performance-Zusammenfassung
if performance_metrics:
    total_workflow_time = sum(sum(times) for times in performance_metrics.values() if times)
    print(f"\n⏱️  GESAMTZEIT: {total_workflow_time:.3f} Sekunden")
    
    # Durchsatz berechnen
    operations_count = sum(len(times) for times in performance_metrics.values())
    throughput = operations_count / total_workflow_time if total_workflow_time > 0 else 0
    print(f"📈 DURCHSATZ: {throughput:.2f} Operationen/Sekunde")

print("\n" + "="*70)
print("✅ Self-Sovereign Identity Workflow demonstriert!")
print("="*70)

print("\n🎓 Was wurde demonstriert:")
print("  1. ✅ Ledger-Setup mit DIDs und Rollen")
print("  2. ✅ Schema und Credential Definition auf Ledger")
print("  3. ✅ Sichere Peer-to-Peer Connections (DIDComm)")
print("  4. ✅ Verifiable Credential Issuance")
print("  5. ✅ Zero-Knowledge Proof Presentation")
print("  6. ✅ Cryptographic Verification")
print("  7. ✅ Performance-Messung und -Analyse")

print("\n💡 Nächste Schritte:")
print("  • Revocation-Support implementieren")
print("  • Mehr Credentials ausstellen")
print("  • Komplexere Proof-Requests mit mehreren Credentials")
print("  • Performance-Optimierungen testen")
print("  • Production-Deployment vorbereiten")

In [ ]:
# Cell 34: Export Performance-Daten (Optional)

print("💾 Performance-Daten Export\n")

# Erstelle Export-Dictionary
export_data = {
    "timestamp": datetime.now().isoformat(),
    "workflow": "SSI Complete Workflow",
    "agents": {
        "issuer": ISSUER_ADMIN_URL,
        "holder": HOLDER_ADMIN_URL,
        "verifier": VERIFIER_ADMIN_URL
    },
    "performance_metrics": {}
}

# Bereite Performance-Metriken für Export vor
for operation, times in performance_metrics.items():
    if times:
        export_data["performance_metrics"][operation] = {
            "count": len(times),
            "average_seconds": sum(times) / len(times),
            "min_seconds": min(times),
            "max_seconds": max(times),
            "total_seconds": sum(times),
            "measurements": times
        }

# Berechne Gesamtstatistiken
total_time = sum(sum(times) for times in performance_metrics.values() if times)
export_data["total_workflow_time_seconds"] = total_time

# Als JSON speichern (optional)
import json
from pathlib import Path

output_file = Path("ssi_workflow_performance.json")
with open(output_file, "w") as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Performance-Daten exportiert nach: {output_file}")
print(f"   Dateigröße: {output_file.stat().st_size} Bytes")

# Zeige Vorschau
print("\n📋 Export-Vorschau:")
print(f"   Workflow: {export_data['workflow']}")
print(f"   Timestamp: {export_data['timestamp']}")
print(f"   Operationen: {len(export_data['performance_metrics'])}")
print(f"   Gesamtzeit: {export_data['total_workflow_time_seconds']:.3f}s")